In [ ]:
from nlb_tools.nwb_interface import NWBDataset
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib as mpl
from sklearn.linear_model import Ridge, LinearRegression, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold
import math
import multiprocessing as mp
from scipy.linalg import orth
import scipy.stats

from Neural_Decoding.preprocessing_funcs import get_spikes_with_history
from Area2_analysis.lr_funcs import process_train_test, gaussian_filter1d_oneside, comp_cc, xcorr, r2_score
from Area2_analysis.lr_funcs import get_sses_pred, get_sses_mean, nans
from Area2_analysis.lr_funcs import fit_and_predict, sub_and_predict, pred_with_new_weights
from Area2_analysis.lr_funcs import fit_and_predict_lasso, sub_and_predict_lasso
from Area2_analysis.lr_funcs import calc_proj, principal_angles, angle_between

matplotlib.rc('font', size=18)


In [2]:
import importlib
import Area2_analysis.lr_funcs
importlib.reload(Area2_analysis.lr_funcs)
from Area2_analysis.lr_funcs import fit_and_predict_DNN, fit_and_predict_weighted

In [3]:
figDir = '/Users/sherryan/area2_population_analysis/sfn/'

# Load data

In [ ]:
foldername = "~/area2_population_analysis/s1-kinematics/actpas_NWB/"
# monkey = "Han_20171207"
# filename = foldername + monkey + "_COactpas_TD_offset2.nwb"

# monkey = "Chips_20170913"
# filename = foldername + monkey + "_COactpas_TD.nwb"

monkey = 'Duncan_20190710'
filename = foldername + monkey + "_COactpas_offset2.nwb"

dataset_10ms = NWBDataset(filename, split_heldout=False)

dataset_10ms.resample(10) #in 10-ms bin, has to resample first for Duncan
bin_width = dataset_10ms.bin_width
print(bin_width)


# xyz_force = np.array([dataset_5ms.data['force']['x'].to_numpy(), dataset_5ms.data['force']['y'].to_numpy(), dataset_5ms.data['force']['z'].to_numpy()]).T
# dataset_10ms.add_continuous_data(xyz_force,'manip_force',chan_names = ['x','y','z'])

# dataset_10ms.smooth_spk(40, name='smth_40')
# dataset_10ms.smooth_spk(20, name='smth_20')

# gaussian_kernel_width = 150 #in ms
# sigma = int(gaussian_kernel_width/bin_width)
# data_smoothed = gaussian_filter1d_oneside(dataset_10ms.data.spikes.to_numpy().astype(np.float64),sigma,axis=0)
# dataset_10ms.add_continuous_data(data_smoothed,'spikes_smth_150_oneside')

In [ ]:
xy_vel = dataset_10ms.data['hand_vel'].to_numpy()
xy_acc = np.diff(xy_vel, axis = 0, prepend=[xy_vel[0]])
dataset_10ms.add_continuous_data(xy_acc,'hand_acc',chan_names = ['x','y'])

In [ ]:
filename = '/Users/sherryan/area2_population_analysis/s1-kinematics/'+monkey+'_COactpas_with_emg_TD.mat'
import scipy.io
mat = scipy.io.loadmat(filename)
EMG = mat['trial_data']['emg'][0,0]
dataset_10ms.add_continuous_data(EMG,'EMG')

In [ ]:
mat['trial_data']['emg_names'][0,0]

In [ ]:
all_data = dataset_10ms.data['EMG'].to_numpy()
print(all_data.shape)
data_for_pca = all_data[~np.isnan(all_data).any(axis=1)]
print(data_for_pca.shape)
explained_var = []
for n in range(20):
    scaler = StandardScaler()
    X = scaler.fit_transform(data_for_pca)
    pca = PCA(n_components=n)
    X = pca.fit(X)
    explained_var.append(np.sum(pca.explained_variance_ratio_))
plt.plot(range(20),explained_var)
print(explained_var)

In [ ]:
all_data = dataset_10ms.data['EMG'].to_numpy()
print(all_data.shape)
data_for_pca = all_data[~np.isnan(all_data).any(axis=1)]
print(data_for_pca.shape)
n_dims = 10
scaler = StandardScaler()
X = scaler.fit_transform(data_for_pca)
pca = PCA(n_components=n_dims,random_state = 42)
X = pca.fit(X)
scaler = StandardScaler()
X = scaler.fit_transform(data_for_pca)
pca = PCA(n_components=n_dims,random_state = 42)
X = pca.fit(X)
    
PCA_data = nans([all_data.shape[0],n_dims])
idx = 0
for dp in all_data:
    dp = dp.reshape((1, -1))
    if np.isnan(dp).any():
        dp_pca = nans([1,n_dims])
    else:
        dp_pca = pca.transform(scaler.transform(dp))
    PCA_data[idx,:] = dp_pca
    idx+=1
print(PCA_data.shape)
dataset_10ms.add_continuous_data(PCA_data,'EMG_PCA')
print('PCA total var explained:',sum(pca.explained_variance_ratio_))

In [ ]:
muscle_len = dataset_10ms.data['muscle_len'].to_numpy()
muscle_vel = dataset_10ms.data['muscle_vel'].to_numpy()
joint_ang = dataset_10ms.data['joint_ang'].to_numpy()
joint_vel = dataset_10ms.data['joint_vel'].to_numpy()

In [ ]:
all_data = np.hstack([muscle_len,muscle_vel])
print(all_data.shape)
data_for_pca = all_data[~np.isnan(all_data).any(axis=1)]
print(data_for_pca.shape)
explained_var = []
for n in range(20):
    scaler = StandardScaler()
    X = scaler.fit_transform(data_for_pca)
    pca = PCA(n_components=n)
    X = pca.fit(X)
    explained_var.append(np.sum(pca.explained_variance_ratio_))
plt.plot(range(20),explained_var)
plt.title('muscle len+vel')
print(explained_var)

In [ ]:
all_data = np.hstack([joint_ang,joint_vel])
print(all_data.shape)
data_for_pca = all_data[~np.isnan(all_data).any(axis=1)]
print(data_for_pca.shape)
explained_var = []
for n in range(14):
    scaler = StandardScaler()
    X = scaler.fit_transform(data_for_pca)
    pca = PCA(n_components=n)
    X = pca.fit(X)
    explained_var.append(np.sum(pca.explained_variance_ratio_))
plt.plot(range(14),explained_var)
plt.title('joint ang+vel')
print(explained_var)

In [ ]:
all_data = np.hstack([muscle_len,muscle_vel])
print(all_data.shape)
data_for_pca = all_data[~np.isnan(all_data).any(axis=1)]
print(data_for_pca.shape)
n_dims = 10
scaler = StandardScaler()
X = scaler.fit_transform(data_for_pca)
pca = PCA(n_components=n_dims,random_state = 42)
X = pca.fit(X)
scaler = StandardScaler()
X = scaler.fit_transform(data_for_pca)
pca = PCA(n_components=n_dims,random_state = 42)
X = pca.fit(X)
    
PCA_data = nans([all_data.shape[0],n_dims])
idx = 0
for dp in all_data:
    dp = dp.reshape((1, -1))
    if np.isnan(dp).any():
        dp_pca = nans([1,n_dims])
    else:
        dp_pca = pca.transform(scaler.transform(dp))
    PCA_data[idx,:] = dp_pca
    idx+=1
print(PCA_data.shape)
dataset_10ms.add_continuous_data(PCA_data,'muscle_PCA')
print('PCA total var explained:',sum(pca.explained_variance_ratio_))

In [ ]:
all_data = np.hstack([joint_ang,joint_vel])
print(all_data.shape)
data_for_pca = all_data[~np.isnan(all_data).any(axis=1)]
print(data_for_pca.shape)
n_dims = 10
scaler = StandardScaler()
X = scaler.fit_transform(data_for_pca)
pca = PCA(n_components=n_dims,random_state = 42)
X = pca.fit(X)
scaler = StandardScaler()
X = scaler.fit_transform(data_for_pca)
pca = PCA(n_components=n_dims,random_state = 42)
X = pca.fit(X)
    
PCA_data = nans([all_data.shape[0],n_dims])
idx = 0
for dp in all_data:
    dp = dp.reshape((1, -1))
    if np.isnan(dp).any():
        dp_pca = nans([1,n_dims])
    else:
        dp_pca = pca.transform(scaler.transform(dp))
    PCA_data[idx,:] = dp_pca
    idx+=1
print(PCA_data.shape)
dataset_10ms.add_continuous_data(PCA_data,'joint_PCA')
print('PCA total var explained:',sum(pca.explained_variance_ratio_))

In [ ]:
n_dims = 20 # for PCA

# trial_mask = active_mask
n_trials = dataset_10ms.trial_info.shape[0]
print(n_trials,'total trials')
n_neurons = dataset_10ms.data.spikes.shape[1]
print(n_neurons,'neurons')


all_data = np.array(dataset_10ms.data.spikes_smth_40)
print(all_data.shape)
if not np.isnan(all_data).any():
    scaler = StandardScaler()
    X = scaler.fit_transform(all_data)
    pca = PCA(n_components=n_dims,random_state = 42)
    PCA_data = pca.fit_transform(X)
print(PCA_data.shape)
dataset_10ms.add_continuous_data(X,'spikes_smth_40_zscored')
dataset_10ms.add_continuous_data(PCA_data,'PCA_40')
print('PCA total var explained:',sum(pca.explained_variance_ratio_))


In [ ]:
#make dictionary for trial condition (reaching directions) for Stratified CV
dataset = dataset_10ms
active_mask = (dataset.trial_info.ctr_hold_bump==0) & (dataset.trial_info['split'] != 'none')
passive_mask = (dataset.trial_info.ctr_hold_bump==1) & (dataset.trial_info['split'] != 'none')
nan_mask = (np.isnan(dataset.trial_info.ctr_hold_bump)) & (dataset.trial_info['split'] != 'none')
all_mask = (dataset.trial_info['split'] != 'none')

trial_mask = all_mask
valid_n_trials = dataset.trial_info.loc[trial_mask].shape[0]
print(valid_n_trials,'valid trials')

trial_mask = active_mask
active_trials_idx = np.array(dataset.trial_info.loc[trial_mask]['trial_id'])
active_n_trials = dataset.trial_info.loc[trial_mask].shape[0]
print(active_n_trials,'active trials')

trial_mask = passive_mask
passive_trials_idx = np.array(dataset.trial_info.loc[trial_mask]['trial_id'])
passive_n_trials = dataset.trial_info.loc[trial_mask].shape[0]
print(passive_n_trials,'passive trials')

trial_mask = nan_mask
nan_trials_idx = np.array(dataset.trial_info.loc[trial_mask]['trial_id'])
nan_n_trials = dataset.trial_info.loc[trial_mask].shape[0]
print(nan_n_trials,'reach bump trials')

active_cond_dir_idx = []
passive_cond_dir_idx = []
nan_cond_dir_idx = []
nan_bump_cond_dir_idx = []
for direction in [0,45,90,135,180,225,270,315]:
# for direction in [0,90,180,270]:
    active_cond_dir_idx.append(np.where((dataset.trial_info['cond_dir']%360 == direction) & (dataset.trial_info['ctr_hold_bump'] == 0) & \
           (dataset.trial_info['split'] != 'none'))[0])
    passive_cond_dir_idx.append(np.where((dataset.trial_info['cond_dir']%360 == direction) & (dataset.trial_info['ctr_hold_bump'] == 1) & \
           (dataset.trial_info['split'] != 'none'))[0])
    nan_cond_dir_idx.append(np.where((dataset.trial_info['cond_dir']%360 == direction) & (np.isnan(dataset.trial_info.ctr_hold_bump)) & \
           (dataset.trial_info['split'] != 'none'))[0])
    nan_bump_cond_dir_idx.append(np.where((dataset.trial_info['bump_dir']%360 == direction) & (np.isnan(dataset.trial_info.ctr_hold_bump)) & \
           (dataset.trial_info['split'] != 'none'))[0])

active_cond_dict = nans([active_n_trials])
i = 0
for idx in active_trials_idx:
    for cond in range(0,len(active_cond_dir_idx)):
        if idx in active_cond_dir_idx[cond]:
            active_cond_dict[i] = cond
            break
    i+=1
print(active_cond_dict)
print(len(active_cond_dict))

passive_cond_dict = nans([passive_n_trials])
i = 0
for idx in passive_trials_idx:
    for cond in range(0,len(passive_cond_dir_idx)):
        if idx in passive_cond_dir_idx[cond]:
            passive_cond_dict[i] = cond
            break
    i+=1
print(passive_cond_dict)
print(len(passive_cond_dict))

nan_cond_dict = nans([nan_n_trials])
i = 0
for idx in nan_trials_idx:
    for cond in range(0,len(nan_cond_dir_idx)):
        if idx in nan_cond_dir_idx[cond]:
            nan_cond_dict[i] = cond
            break
    i+=1
print(nan_cond_dict)
print(len(nan_cond_dict))

nan_bump_cond_dict = nans([nan_n_trials])
i = 0
for idx in nan_trials_idx:
    for cond in range(0,len(nan_bump_cond_dir_idx)):
        if idx in nan_bump_cond_dir_idx[cond]:
            nan_bump_cond_dict[i] = cond
            break
    i+=1
print(nan_bump_cond_dict)
print(len(nan_bump_cond_dict))

In [ ]:
passive_df = dataset_10ms.make_trial_data(align_field='move_onset_time', align_range = (-100,100), ignored_trials = ~passive_mask)
set(passive_trials_idx) - set(passive_df['trial_id'].unique())

In [ ]:
active_df = dataset_10ms.make_trial_data(align_field='move_onset_time', align_range = (-100,100), ignored_trials = ~active_mask)
del_indices = list(set(active_trials_idx) - set(active_df['trial_id'].unique()))
print('was',active_n_trials,'active trials')
active_n_trials = active_n_trials - len(list(set(active_trials_idx) - set(active_df['trial_id'].unique())))
active_cond_dict = np.delete(active_cond_dict,np.argwhere(active_trials_idx==del_indices)[0])
print('now',active_n_trials,'active trials')
print(len(active_cond_dict))

In [ ]:
from sca.models import SCA
align_range = (-100, 1000)
active_trial_data = dataset_10ms.make_trial_data(align_field='move_onset_time', align_range=align_range, ignored_trials=~active_mask)
active_trial_spsm = np.array(active_trial_data.spikes_smth_40)
target_n_trials = active_trial_data['trial_id'].nunique()
n_timepoints = int((align_range[1]-align_range[0])/bin_width)
active_sample_weights= np.ones((target_n_trials, n_timepoints))
# active_sample_weights[:,:int(100/dataset_10ms.bin_width)] = 10
active_sample_weights = active_sample_weights.flatten()
print(active_sample_weights.shape)

align_range = (-100, 500)
passive_trial_data = dataset_10ms.make_trial_data(align_field='move_onset_time', align_range=align_range, ignored_trials=~passive_mask)
passive_trial_spsm = np.array(passive_trial_data.spikes_smth_40)
target_n_trials = passive_trial_data['trial_id'].nunique()
n_timepoints = int((align_range[1]-align_range[0])/bin_width)
passive_sample_weights= np.ones((target_n_trials, n_timepoints))
passive_sample_weights = passive_sample_weights.flatten()
print(passive_sample_weights.shape)

sample_weights = np.hstack((active_sample_weights, passive_sample_weights))
print(sample_weights.shape)
all_trial_spsm = np.concatenate((active_trial_spsm, passive_trial_spsm),axis=0)
print(all_trial_spsm.shape)

all_data = np.array(dataset_10ms.data.spikes_smth_40)
print(all_data.shape)
if not np.isnan(all_trial_spsm).any():
    scaler = StandardScaler()
    X_trial = scaler.fit_transform(all_trial_spsm,sample_weight=sample_weights)
    sca = SCA(n_components=n_dims)
    sca.fit(X_trial) # scaler and sca fit to trial data
    X_all = scaler.transform(all_data) #scaler and sca transform all data
    SCA_data = sca.transform(X_all)
print(SCA_data.shape)
dataset_10ms.add_continuous_data(SCA_data,'SCA_40')
print('SCA_40 var explained:',sca.r2_score)

ssa_order_smth40 = np.argsort(-np.array(sca.explained_squared_activity))
print('SCA_40 activity explained:',sca.explained_squared_activity[ssa_order_smth40])


# Simulation

In [ ]:
trial_range = [-100, 1000]
lag_range = [-300, 300]
kin_range = [trial_range[0] + lag_range[0] + (-200), trial_range[1] + lag_range[1] + (+200)]                                
lag_axis = np.arange(lag_range[0], lag_range[1]+1, 10)
nrn_axis = np.arange(trial_range[0]+lag_range[0], trial_range[1]+lag_range[1]+1, 10)
# To predict trial_range, we need wider neural_range, which requires wider kin_range

df = dataset_10ms.make_trial_data(align_field='move_onset_time', align_range=kin_range, ignored_trials=~active_mask, allow_overlap=True)
n_trials = df['trial_id'].nunique()
# acc_array = df['hand_acc'].to_numpy().reshape(active_n_trials, -1, 2)
vel_array = df['hand_vel'].to_numpy().reshape(active_n_trials, -1, 2)

kin_axis = np.arange(kin_range[0], kin_range[1]+1, dataset_10ms.bin_width)
print('neural axis',nrn_axis[0], nrn_axis[-1])
print('kinematics axis',kin_axis[0], kin_axis[-1])

In [ ]:
n_runs = 50
ang_runs = nans([n_runs])
r2_runs = nans([n_runs, len(lag_axis)])
cd_r2_arr = nans([n_runs, len(lag_axis)])
fb_r2_arr = nans([n_runs, len(lag_axis)])
cd_lag = -50
fb_lag = 50
noise_level = 15

for b in range(n_runs):
    mu, sigma = 0,1
    n_nrn = 50

    # Random We and Wa
    nrn_weight_0 = np.random.normal(mu, sigma, size = (n_nrn,1)) 
    nrn_weight_1 = np.random.normal(mu, sigma, size = (n_nrn,1))
    # nrn_weight = np.concatenate([nrn_weight_0, nrn_weight_1], axis = 1)
    nrn_weight = orth(np.concatenate([nrn_weight_0, nrn_weight_1], axis = 1))

    # Random We = Wa
    # nrn_weight_0 = np.random.normal(mu, sigma, size = (n_nrn,1))
    # nrn_weight = np.tile(nrn_weight_0,(1,2)) 

    # Random We = -Wa
    # nrn_weight_0 = np.random.normal(mu, sigma, size = (n_nrn,1))
    # nrn_weight = np.concatenate([nrn_weight_0, -nrn_weight_0], axis = 1) 
    
    n_bins = len(nrn_axis)
    nrn_activity = nans([n_trials, n_bins, n_nrn])
    cd_nrn_activity = nans([n_trials, n_bins, n_nrn])
    fb_nrn_activity = nans([n_trials, n_bins, n_nrn])
    for i in range(n_bins):
        cd_signal = np.outer(vel_array[:, i+np.argwhere(kin_axis==nrn_axis[0]+(-cd_lag))[0,0], 1], nrn_weight[:,0])
        fb_signal = np.outer(vel_array[:, i+np.argwhere(kin_axis==nrn_axis[0]+(-fb_lag))[0,0], 1], nrn_weight[:,1])
        cd_nrn_activity[:,i,:] = cd_signal
        fb_nrn_activity[:,i,:] = fb_signal
        nrn_activity[:,i,:] = cd_signal+fb_signal
    
    # nrn_activity_flat = nrn_activity_flat.reshape(-1,n_nrn)
    # cd_nrn_activity_flat = cd_nrn_activity.reshape(-1,n_nrn)
    # cd_sig_noise = np.zeros((n_nrn,n_nrn))
    # np.fill_diagonal(cd_sig_noise, np.std(cd_nrn_activity_flat,axis=0))
    # cd_noise = np.random.multivariate_normal(np.zeros(n_nrn), cd_sig_noise, cd_nrn_activity_flat.shape[0]) * noise_percentage
    # fb_nrn_activity_flat = fb_nrn_activity.reshape(-1,n_nrn)
    # fb_sig_noise = np.zeros((n_nrn,n_nrn))
    # np.fill_diagonal(fb_sig_noise, np.std(fb_nrn_activity_flat,axis=0))
    # fb_noise = np.random.multivariate_normal(np.zeros(n_nrn), fb_sig_noise, fb_nrn_activity_flat.shape[0]) * noise_percentage
    # noisy_nrn_activity = (nrn_activity_flat+cd_noise+fb_noise).reshape(nrn_activity.shape)

    nrn_activity_flat = nrn_activity.reshape(-1,n_nrn)
    sig_noise = np.zeros((n_nrn,n_nrn))
    np.fill_diagonal(sig_noise,1)
    noise = np.random.multivariate_normal(np.zeros(n_nrn), sig_noise, nrn_activity_flat.shape[0]) * noise_level
    noisy_nrn_activity = (nrn_activity_flat+noise).reshape(nrn_activity.shape)

    cd_nrn_activity_flat = cd_nrn_activity.reshape(-1,n_nrn)
    noisy_cd_nrn_activity = (cd_nrn_activity_flat+noise).reshape(cd_nrn_activity.shape)

    fb_nrn_activity_flat = fb_nrn_activity.reshape(-1,n_nrn)
    noisy_fb_nrn_activity = (fb_nrn_activity_flat+noise).reshape(fb_nrn_activity.shape)    

    coefs_arr = nans([len(lag_axis), n_nrn])
    r2_arr = nans([len(lag_axis)]); 
    cd_r2 = nans([len(lag_axis)]); fb_r2 = nans([len(lag_axis)])
    y = vel_array[:,np.argwhere(kin_axis==trial_range[0])[0,0]:np.argwhere(kin_axis==trial_range[1])[0,0],1]
    y_reshaped = y.reshape(-1,1)
    for j in range(len(lag_axis)):
        start, end = int(0+j), int(y.shape[1]+j)
        
        X = noisy_cd_nrn_activity[:,start:end,:]
        X = X.reshape((X.shape[0]* X.shape[1]),X.shape[2])
        lr =  LinearRegression().fit(X, y_reshaped)
        cd_r2[j] = lr.score(X, y_reshaped)

        X = noisy_fb_nrn_activity[:,start:end,:]
        X = X.reshape((X.shape[0]* X.shape[1]),X.shape[2])
        lr =  LinearRegression().fit(X, y_reshaped)
        fb_r2[j] = lr.score(X, y_reshaped)

        X = noisy_nrn_activity[:,start:end,:]
        X = X.reshape((X.shape[0]* X.shape[1]),X.shape[2])
        lr =  LinearRegression().fit(X, y_reshaped)
        coefs_arr[j] = lr.coef_
        r2_arr[j] = lr.score(X, y_reshaped)

    ang_runs[b] = math.degrees(angle_between(coefs_arr[np.argwhere(lag_axis==cd_lag)[0,0],:],coefs_arr[np.argwhere(lag_axis==fb_lag)[0,0],:])) 
    r2_runs[b,:] = r2_arr; 
    cd_r2_arr[b,:] = cd_r2; fb_r2_arr[b,:] = fb_r2
print(r2_runs)


In [ ]:
fig, ax = plt.subplots(figsize=(4,3))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

tr_idx = 0
nrn_idx = 12

plt.plot(nrn_axis,cd_nrn_activity[tr_idx,:,nrn_idx],color = 'green',label = 'cd signal')
plt.plot(nrn_axis,fb_nrn_activity[tr_idx,:,nrn_idx],color = 'magenta',label = 'fb signal')
plt.plot(nrn_axis,nrn_activity[tr_idx,:,nrn_idx],color = 'brown',label = 'sum signal')
plt.plot(nrn_axis,noisy_nrn_activity[tr_idx,:,nrn_idx],color = 'gray',alpha=0.8,label = 'noisy signal')
plt.legend(fontsize=8)
plt.axvline(0,color = 'k', linestyle='--')
plt.tight_layout()
# plt.savefig(figDir + monkey+'_example_nrn_low_noise.pdf',dpi = 'figure')
plt.show()

In [ ]:
print(np.max(r2_runs))
fig, ax = plt.subplots(figsize=(4,3))

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.plot(lag_axis, np.mean(r2_runs,axis=0), color = 'brown')
plt.plot(lag_axis, np.mean(cd_r2_arr,axis=0), color = 'green')
plt.plot(lag_axis, np.mean(fb_r2_arr,axis=0), color = 'magenta')
plt.axvline(cd_lag, color = 'k', linestyle='--')
plt.axvline(fb_lag, color = 'k', linestyle='--')
plt.xlabel("Time lag (ms)")
plt.ylabel('R2')
plt.ylim([-0.05,1.05])
plt.tight_layout()
# plt.savefig(figDir + monkey + '_sim_r2_low_noise.pdf', dpi = 'figure')
plt.show()
plt.hist(ang_runs)
plt.xlabel("Angle (deg)")
plt.ylabel('Frequency')

# Plot Variables

## Correlations

In [ ]:
df['ac2_CD_proj_spikes_smth_40_oneside'].to_numpy().shape

In [ ]:
avg_c_x.shape

In [ ]:
np.arange(-300, 300, 10)[np.argmax(avg_c_x1)]

In [ ]:
## Cross-correlation
df = dataset_10ms.make_trial_data(align_field='move_onset_time', align_range = (-100,1000), ignored_trials = ~active_mask)
n_trials = df['trial_id'].nunique()

cd_array = df['ac2_CD_proj_spikes_smth_40_oneside'].to_numpy().reshape(n_trials, -1, 3)

# pos_array = df['hand_pos'].to_numpy().reshape(n_trials, -1, 2)
# acc_array = df['hand_acc'].to_numpy().reshape(n_trials, -1, 2)
vel_array = df['hand_vel'].to_numpy().reshape(n_trials, -1, 2)
maxlags = 30 # times binsize is in ms (300ms for best display)
X = cd_array
Y = vel_array

print(X.shape)
cc_arr = nans([n_trials, maxlags*2+1])
for i in range(n_trials):
    x = X[i,:,0]
    y = Y[i,:,0]
    lags, c = xcorr(x, y, maxlags)
    cc_arr[i,:] = c
avg_c_x = np.mean(cc_arr, axis=0)

cc_arr = nans([n_trials, maxlags*2+1])
for i in range(n_trials):
    x = X[i,:,1]
    y = Y[i,:,0]
    lags, c = xcorr(x, y, maxlags)
    cc_arr[i,:] = c
avg_c_x1 = np.mean(cc_arr, axis=0)


cc_arr = nans([n_trials, maxlags*2+1])
for i in range(n_trials):
    x = X[i,:,2]
    y = Y[i,:,1]
    lags, c = xcorr(x, y, maxlags)
    cc_arr[i,:] = c
avg_c_y = np.mean(cc_arr, axis=0)

x_axis = lags*dataset_10ms.bin_width
fig, ax = plt.subplots()
ax.plot(x_axis, avg_c_x, color = 'green', label = 'x1')
ax.plot(x_axis, avg_c_x1, color = 'green', label = 'x2')

ax.plot(x_axis, avg_c_y, color = 'blue', label = 'y')
tmax = x_axis[int(np.mean((np.argmax(abs(avg_c_x)),np.argmax(abs(avg_c_y)))))]
# ax.axvline(tmax, color = 'k',linestyle = '--')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
print(tmax)
plt.legend()
plt.xlabel('Time lag (ms)')
plt.ylabel('Normalized cross-corr')
plt.title('CD dims vs. hand vel')

In [ ]:
## Autocorrelation
df = dataset_10ms.make_trial_data(align_field='move_onset_time', align_range = (-100,1000), ignored_trials = ~active_mask)
n_trials = df['trial_id'].nunique()

vel_array = df['hand_vel'].to_numpy().reshape(n_trials, -1, 2)
maxTimeLag = 500 #in ms
X = vel_array

print(X.shape)
binSize = dataset_10ms.bin_width
numBin = X.shape[1]
x1 = X[:,:,0]
x2 = X[:,:,0]
ac_x = comp_cc(x1,x2,maxTimeLag,binSize,numBin)

x1 = X[:,:,1]
x2 = X[:,:,1]
ac_y = comp_cc(x1,x2,maxTimeLag,binSize,numBin)

time_axis = np.arange(0, maxTimeLag, binSize)
fig, ax = plt.subplots()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.plot(time_axis,ac_x/ac_x[0],color = 'green', label = 'x')
ax.plot(time_axis,ac_y/ac_y[0],color = 'blue', label = 'y')
plt.legend()
plt.xlabel("Time lag (ms)")
plt.ylabel("Normalized autocorrelation")
plt.tight_layout()
# plt.savefig(figDir + monkey + '_autocorrelation_vel.pdf',dpi = 'figure')

## PCA's

In [ ]:
plot_dir = np.array([0,45,90,135,180,225,270,315]) 
directions = np.array([0,45,90,135,180,225,270,315])

# plot_dir = np.array([0,90,180,270]) 
# directions = np.array([0,90,180,270])

cmap = plt.get_cmap('coolwarm',len(plot_dir))
custom_palette = [mpl.colors.rgb2hex(cmap(i)) for i in range(len(plot_dir))]

In [ ]:
# Plot PCA projections over trial, for different reaching directions
plot_field = 'SCA_40'
n_dims = dataset.data[plot_field].shape[1]
# order = np.arange(n_dims)
order = ssa_order_smth40

pred_range = (-100, 1000)
trial_mask = active_mask
cond_dict = active_cond_dict
n_timepoints = int((pred_range[1] - pred_range[0])/dataset_10ms.bin_width)
data = dataset_10ms.make_trial_data(align_field='move_onset_time', align_range=pred_range, ignored_trials=~trial_mask)
n_trials = data['trial_id'].nunique()
trials_pca = nans([n_trials,n_timepoints,n_dims])
i = 0
for idx, trial in data.groupby('trial_id'):
    trials_pca[i,:,:]=trial[plot_field].to_numpy()
    i+=1
print(trials_pca.shape)

x_axis = np.arange(pred_range[0], pred_range[1], dataset_10ms.bin_width)
# define some useful time points
move_idx=0
ret_idx = 500

plot_dims = 10

fig,ax=plt.subplots(plot_dims,1,figsize=(10,15))
for i in range(plot_dims):
    for j in range(len(plot_dir)):
        color = custom_palette[j]
        dir_idx = np.argwhere(directions == plot_dir[j])[0]
        cond_mean_proj = np.mean(trials_pca[np.argwhere(cond_dict==dir_idx).flatten(),:,:], axis = 0)[:,order[i]] 
        pca_mean = np.mean(data[plot_field].to_numpy(),axis = 0)[order[i]] 
        ax[i].plot(x_axis,cond_mean_proj - pca_mean,linewidth=2.25,color = color,label = plot_dir[j])
        
        ax[i].axvline(move_idx, color='k',linewidth = .5)
        ax[i].axvline(ret_idx, color='k',linewidth = .5)
        ax[i].set_xlim([-100,1000])
        # ax[i].set_ylim([-15, 15])
        ax[i].axhline(0,color ='k',ls = '--')
        if i<plot_dims-1:
            ax[i].set_xticks([])
        else:
            ax[i].set_xlabel('Time after movement onset (ms)')
            
        # ax[i].set_yticks([])
        ax[i].set_ylabel('Dim. '+str(i+1))

    ax[0].set_title('Active trials ' + plot_field)
    
plt.legend(bbox_to_anchor = (1, 1), loc = 'upper left')
plt.tight_layout()
# plt.savefig(figDir + monkey + '_PCA_active.pdf',dpi = 'figure')

In [ ]:
# Plot PCA projections over trial, for different reaching directions
order = np.arange(n_dims)
# order = ssa_order_smth40

pred_range = (-100, 500)
trial_mask = passive_mask
cond_dict = passive_cond_dict
n_timepoints = int((pred_range[1] - pred_range[0])/dataset_10ms.bin_width)
# n_trials = dataset_10ms.trial_info.loc[trial_mask].shape[0]
data = dataset_10ms.make_trial_data(align_field='move_onset_time', align_range=pred_range, ignored_trials=~trial_mask)
n_trials = data['trial_id'].nunique()
trials_pca = nans([n_trials,n_timepoints,n_dims])
i = 0
for idx, trial in data.groupby('trial_id'):
    trials_pca[i,:,:]=trial[plot_field].to_numpy()
    i+=1
print(trials_pca.shape)

x_axis = np.arange(pred_range[0], pred_range[1], dataset_10ms.bin_width)
# define some useful time points
move_idx=0
ret_idx = 200

plot_dims = 10

fig,ax=plt.subplots(plot_dims,1,figsize=(10,15))
for i in range(plot_dims):
    for j in range(len(plot_dir)):
        color = custom_palette[j]
        dir_idx = np.argwhere(directions == plot_dir[j])[0]
        cond_mean_proj = np.mean(trials_pca[np.argwhere(cond_dict==dir_idx).flatten(),:,:], axis = 0)[:,order[i]] 
        pca_mean = np.mean(data[plot_field].to_numpy(),axis = 0)[order[i]]
        ax[i].plot(x_axis,cond_mean_proj - pca_mean,linewidth=2.25,color = color,label = plot_dir[j])
        
        ax[i].axvline(move_idx, color='k',linewidth = .5)
        ax[i].axvline(ret_idx, color='k',linewidth = .5)
        ax[i].set_xlim([-100,500])
        # ax[i].set_ylim([-15, 15])
        ax[i].axhline(0,color ='k',ls = '--')
        if i<plot_dims-1:
            ax[i].set_xticks([])
        else:
            ax[i].set_xlabel('Time after movement onset (ms)')
            
        # ax[i].set_yticks([])
        ax[i].set_ylabel('Dim. '+str(i+1))

    ax[0].set_title('Passive trials ' + plot_field)
    
plt.legend(bbox_to_anchor = (1, 1), loc = 'upper left')
plt.tight_layout()
# plt.savefig(figDir + monkey + '_PCA_passive.pdf',dpi = 'figure')

## Single-neurons

In [ ]:
# #Get a specific neuron's original index from GLM results to plot
# with np.load(monkey+'_hf_neuron_filter.npz') as data:
#     neuron_filter = data['neuron_filter']
# fr_filtered_idx = np.argwhere(neuron_filter==1).flatten()
# index_in_glm = [18, 27, 34, 35, 42, 48]

# index_original = fr_filtered_idx[index_in_glm]
# print(index_original)

In [ ]:
plot_dir = np.array([0,45,90,135,180,225,270,315]) 
directions = np.array([0,45,90,135,180,225,270,315])

# plot_dir = np.array([0,90,180,270]) 
# directions = np.array([0,90,180,270])

cmap = plt.get_cmap('coolwarm',len(plot_dir))
custom_palette = [mpl.colors.rgb2hex(cmap(i)) for i in range(len(plot_dir))]

In [ ]:
# Plot single neuron activity over trial, for different reaching directions
dataset = dataset_10ms
nrn_idx = 89
pred_range = (-300, 1000)
trial_mask = active_mask
cond_dict = active_cond_dict
n_timepoints = int((pred_range[1] - pred_range[0])/dataset.bin_width)
n_trials = dataset.trial_info.loc[trial_mask].shape[0]
data = dataset.make_trial_data(align_field='move_onset_time', align_range=pred_range, ignored_trials=~trial_mask)
trials_activity = nans([n_trials,n_timepoints])
i = 0
for idx, trial in data.groupby('trial_id'):
    trials_activity[i,:]=trial.spikes_smth_40.to_numpy()[:,nrn_idx]
    i+=1
print(trials_activity.shape)

plot_dir = np.array([0,45,90,135,180,225,270,315]) 
directions = np.array([0,45,90,135,180,225,270,315])
# plot_dir = np.array([0,90,180,270]) 
# directions = np.array([0,90,180,270]) 
x_axis = np.arange(pred_range[0], pred_range[1], dataset.bin_width)

# define some useful time points
move_idx=0
ret_idx = 500

fig,ax=plt.subplots(1,1,figsize=(8,2))
for j in range(len(plot_dir)):
    color = custom_palette[j]
    dir_idx = np.argwhere(directions == plot_dir[j])[0]
    cond_mean_proj = np.mean(trials_activity[np.argwhere(cond_dict==dir_idx).flatten(),:], axis = 0)
    ax.plot(x_axis,cond_mean_proj/dataset.bin_width*1000,linewidth=2.25,color = color,label = plot_dir[j])

ax.axvline(move_idx,color='k')
ax.axvline(ret_idx,color='k')

ax.set_xlabel('Time after movement onset (ms)')
ax.set_ylabel('Firing rate')
ax.set_title('Active trials')

plt.legend(bbox_to_anchor = (1, 1), loc = 'upper left')
plt.tight_layout()
# plt.savefig(figDir + monkey + '_example_mix_nrn_active.pdf',dpi = 'figure')
plt.show()


In [ ]:
# Plot single neuron activity over trial, for different reaching directions
dataset = dataset_10ms
pred_range = (-100, 500)
trial_mask = passive_mask
cond_dict = passive_cond_dict
n_timepoints = int((pred_range[1] - pred_range[0])/dataset.bin_width)
n_trials = dataset.trial_info.loc[trial_mask].shape[0]
data = dataset.make_trial_data(align_field='move_onset_time', align_range=pred_range, ignored_trials=~trial_mask)
trials_activity = nans([n_trials,n_timepoints])
i = 0
for idx, trial in data.groupby('trial_id'):
    trials_activity[i,:]=trial.spikes_smth_40.to_numpy()[:,nrn_idx]
    i+=1
print(trials_activity.shape)

plot_dir = np.array([0,45,90,135,180,225,270,315]) 
directions = np.array([0,45,90,135,180,225,270,315])
# plot_dir = np.array([0,90,180,270]) 
# directions = np.array([0,90,180,270]) 
x_axis = np.arange(pred_range[0], pred_range[1], dataset.bin_width)

# define some useful time points
move_idx=0
ret_idx = 200

fig,ax=plt.subplots(1,1,figsize=(8,2))
for j in range(len(plot_dir)):
    color = custom_palette[j]
    dir_idx = np.argwhere(directions == plot_dir[j])[0]
    cond_mean_proj = np.mean(trials_activity[np.argwhere(cond_dict==dir_idx).flatten(),:], axis = 0)
    ax.plot(x_axis,cond_mean_proj/dataset.bin_width*1000,linewidth=2.25,color = color,label = plot_dir[j])

ax.axvline(move_idx,color='k')
ax.axvline(ret_idx,color='k')

ax.set_xlabel('Time after movement onset (ms)')
ax.set_ylabel('Firing rate')
ax.set_title('Passive trials')

plt.legend(bbox_to_anchor = (1, 1), loc = 'upper left')
plt.tight_layout()
# plt.savefig(figDir + monkey + '_example_mix_nrn_passive.pdf',dpi = 'figure')
plt.show()

## Timing plots

In [ ]:
plot_range = (-100,1000)
x_axis = np.arange(plot_range[0], plot_range[1], dataset_10ms.bin_width)
active_df = dataset_10ms.make_trial_data(align_field='move_onset_time', align_range=plot_range, ignored_trials=~active_mask, allow_overlap=True)
# passive_df = dataset_10ms.make_trial_data(align_field='move_onset_time', align_range=plot_range, ignored_trials=~passive_mask, allow_overlap=True)
# nan_df = dataset_10ms.make_trial_data(align_field='bump_time', align_range=plot_range, ignored_trials=~nan_mask, allow_overlap=True)

# speed = np.sqrt(np.sum(dataset_10ms.data['hand_vel'][:].T**2,axis=0)).to_numpy().reshape((-1,1))
# dataset_10ms.add_continuous_data(speed,'speed')
# acceleration = np.diff(speed, axis = 0, prepend=[speed[0]])
# dataset_10ms.add_continuous_data(acceleration,'acceleration') #technically change of speed, for timing plots


In [ ]:
df = active_df
var = 'hand_vel'
plot_dir = [0.0, 90.0, 180.0, 270.0] 
# plot_dir = [45.0, 135.0, 225.0, 315.0] 
plot_dim = 'x' # plot x velocity
colors = ['red', 'blue', 'green', 'orange']
fig, ax = plt.subplots(figsize=(10,2))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
for trial_dir, color in zip(plot_dir, colors):
    cond_ids = dataset.trial_info[dataset.trial_info.cond_dir%360 == trial_dir].trial_id
    for _, trial in df[np.isin(df.trial_id, cond_ids)].groupby('trial_id'):
        plt.plot(x_axis, trial[var][plot_dim], color=color, linewidth=0.5)
plt.xlim([-100, 1000])
plt.xlabel('Time after movement offset (ms)')
plt.ylabel('Hand velocity (cm/s)')

In [ ]:
plot_dir = [0.0, 90.0, 180.0, 270.0] # limit plot directions to reduce cluttering
# plot_dir = [45.0, 135.0, 225.0, 315.0] 
plot_dim = 'y' # plot x velocity
colors = ['red', 'blue', 'green', 'orange']
fig, ax = plt.subplots(figsize=(10,2))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
for trial_dir, color in zip(plot_dir, colors):
    cond_ids = dataset.trial_info[dataset.trial_info.cond_dir%360 == trial_dir].trial_id
    for _, trial in df[np.isin(df.trial_id, cond_ids)].groupby('trial_id'):
        plt.plot(x_axis, trial[var][plot_dim], color=color, linewidth=0.5)
plt.xlim([-100, 1000])
plt.xlabel('Time after movement offset (ms)')
plt.ylabel('Hand velocity (cm/s)')

In [ ]:
dataset.trial_info

In [ ]:
# Trial length
dt = (dataset.trial_info.go_cue_time - dataset.trial_info.start_time).dt.total_seconds()*1000
dt[active_mask]
plt.hist(dt[active_mask])
# plt.xlim([-100, 200])

In [ ]:
#PSTH
active_trials_spikes = []
for _, trial in active_df.groupby('trial_id'):
    active_trials_spikes.append(np.sum(trial.spikes,axis=1))
passive_trials_spikes = []
for _, trial in passive_df.groupby('trial_id'):
    passive_trials_spikes.append(np.sum(trial.spikes,axis=1))
fig, ax = plt.subplots(figsize=(10,6))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.plot(x_axis,np.sum(active_trials_spikes,axis = 0)/dataset_10ms.bin_width*1000/len(active_trials_spikes)/n_neurons,"-o",markersize=5, color = 'k',label = 'Active')
ax.plot(x_axis,np.sum(passive_trials_spikes,axis = 0)/dataset_10ms.bin_width*1000/len(passive_trials_spikes)/n_neurons,"-o",markersize=5, color = 'red',label = 'Passive')
# plt.title('Peristimulus aligned to move_onset')
plt.legend()
ax.set_ylabel('Average Firing Rate (sp/s)')
plt.xlabel('Time after movement onset (ms)')
plt.axvline(0, color = 'k',linestyle = '--')    

# active_trials_EMG = []
# for _, trial in active_df.groupby('trial_id'):
#     active_trials_EMG.append(np.sum(trial.EMG,axis=1))
# passive_trials_EMG = []
# for _, trial in passive_df.groupby('trial_id'):
#     passive_trials_EMG.append(np.sum(trial.EMG,axis=1))
# n_muscles = dataset_10ms.data.EMG.shape[1]
# ax1 = ax.twinx()
# ax1.spines['right'].set_visible(False)
# ax1.spines['top'].set_visible(False)
# ax1.plot(x_axis,np.sum(active_trials_EMG,axis = 0)/len(active_trials_EMG)/n_muscles,"--o",markersize=5, color = 'k',alpha = 0.5)
# ax1.plot(x_axis,np.sum(passive_trials_EMG,axis = 0)/len(passive_trials_EMG)/n_muscles,"--o",markersize=5, color = 'red',alpha = 0.5)
# ax1.set_ylabel('Average Muscle Activity')

plt.xlabel('Time after move onset (ms)')
plt.axvline(0, color = 'k',linestyle = '--')    
plt.xlim([-200, 600])
plt.tight_layout()
# plt.savefig(figDir + monkey + '_psth.pdf',dpi = 'figure')

plt.show()


In [ ]:
#PSTH
active_trials_spikes = []
for _, trial in active_df.groupby('trial_id'):
    active_trials_spikes.append(np.sum(trial.spikes,axis=1))
passive_trials_spikes = []
for _, trial in passive_df.groupby('trial_id'):
    passive_trials_spikes.append(np.sum(trial.spikes,axis=1))
fig, ax = plt.subplots(figsize=(10,6))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.plot(x_axis,np.sum(active_trials_spikes,axis = 0)/dataset_10ms.bin_width*1000/len(active_trials_spikes)/n_neurons,"-o",markersize=5, color = 'k',label = 'Active')
ax.plot(x_axis,np.sum(passive_trials_spikes,axis = 0)/dataset_10ms.bin_width*1000/len(passive_trials_spikes)/n_neurons,"-o",markersize=5, color = 'red',label = 'Passive')
# plt.title('Peristimulus aligned to move_onset')
plt.legend()
ax.set_ylabel('Average Firing Rate (sp/s)')
plt.xlabel('Time after movement onset (ms)')
plt.axvline(0, color = 'k',linestyle = '--')    

active_trials_EMG = []
for _, trial in active_df.groupby('trial_id'):
    active_trials_EMG.append(np.sum(trial['joint_ang'],axis=1))
passive_trials_EMG = []
for _, trial in passive_df.groupby('trial_id'):
    passive_trials_EMG.append(np.sum(trial['joint_ang'],axis=1))
n_muscles = dataset_10ms.data['joint_ang'].shape[1]
ax1 = ax.twinx()
ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)
ax1.plot(x_axis,np.sum(active_trials_EMG,axis = 0)/len(active_trials_EMG)/n_muscles,"--o",markersize=5, color = 'k',alpha = 0.5)
ax1.plot(x_axis,np.sum(passive_trials_EMG,axis = 0)/len(passive_trials_EMG)/n_muscles,"--o",markersize=5, color = 'red',alpha = 0.5)
ax1.set_ylabel('Average Joint Angle')

plt.xlabel('Time after movement onset (ms)')
plt.axvline(0, color = 'k',linestyle = '--')    
plt.xlim([-200, 600])
plt.tight_layout()
# plt.savefig(figDir + monkey + '_psth.pdf',dpi = 'figure')

plt.show()


In [ ]:
#Kinematics
var = 'speed'
fig, ax = plt.subplots(figsize=(10,6))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
count = 0
for _, trial in active_df.groupby('trial_id'):
    if len(trial[var]) == len(x_axis):
        plt.plot(x_axis,trial[var], color='k', linewidth=0.5)
        count+=1
print(count,'active trials')
# count = 0
# for _, trial in passive_df.groupby('trial_id'):
#     if len(trial[var]) == len(x_axis):
#         plt.plot(x_axis, trial[var], color='red', linewidth=0.5)
#         count+=1
# print(count,'passive trials')

plt.xlabel('Time after movement onset (ms)')
plt.ylabel('Hand speed (cm/s)')
plt.axvline(0, color = 'k',linestyle = '--')
# plt.title('Speed aligned to move_onset')
# plt.axvline(120, color = 'k',linestyle = '--')
plt.xlim([-2000,500])
# plt.ylim([-3,100])
plt.tight_layout()
# plt.savefig(figDir + monkey + '_speed_whole.pdf',dpi = 'figure')

# CD/FB subspaces

## Define

In [7]:
dataset = dataset_10ms
# x_field = 'PCA_40'
x_field = 'spikes'
# x_field = 'proj_out'
dim = dataset.data[x_field].shape[1]

# align_range = (-200, 1500)
# align_field = 'move_onset_time'
# mask = active_mask
# active_df = dataset.make_trial_data(align_field=align_field, align_range=align_range, ignored_trials=~mask)
# n_trials = active_n_trials
# X = active_df[x_field].to_numpy().reshape((-1, dim))
# print(X.shape)
# mean = np.nanmean(X,axis=0)
# std = np.nanstd(X,axis=0)
# mean.shape

In [ ]:
## Find CD axes
align_range = (-100,0)
align_field = 'move_onset_time'

mask = active_mask
active_df = dataset.make_trial_data(align_field=align_field, align_range=align_range, ignored_trials=~mask)
cond_dict = active_cond_dict
n_trials = active_n_trials
X = active_df[x_field].to_numpy().reshape((-1, dim))
# X = (X - mean)/std
act_trial_spikes = X.reshape((n_trials, -1, dim))
print(act_trial_spikes.shape)
act_trial_mean_activity = np.mean(act_trial_spikes, axis=1)

dirs = [dataset.trial_info[dataset.trial_info.trial_id == i]['cond_dir'] for i in active_df.trial_id]
cos_x = np.array([round(math.cos(math.radians(i)),3) for i in dirs])
sin_y = np.array([round(math.sin(math.radians(i)),3) for i in dirs])
cos_sin = np.array([cos_x, sin_y]).T
print(cos_sin.shape)

# dirs = [dataset.trial_info[dataset.trial_info.trial_id == i]['cond_dir'] for i in active_df.trial_id]
# filter = np.array([x%90==0 for x in dirs])
# n_trials = len(filter)
# act_trial_mean_activity = act_trial_mean_activity[filter[0::act_trial_spikes.shape[1]].squeeze(),:]
# print(act_trial_mean_activity.shape)

# cos_x = np.array([round(math.cos(math.radians(i)),3) for i in dirs])[filter.squeeze()]
# sin_y = np.array([round(math.sin(math.radians(i)),3) for i in dirs])[filter.squeeze()]
# cos_sin = np.array([cos_x, sin_y]).T
# print(cos_sin.shape)

# n_trials = act_trial_mean_activity.shape[0]
# cond_dict = cond_dict[filter[0::act_trial_spikes.shape[1]].squeeze()]

In [ ]:
# def calc_proj_matrix_sparse(A,reg=1e-10):
#     if A.ndim == 1:
#         A = A.reshape(-1, 1)
#     return A @ np.linalg.pinv(A.T @ A + reg * np.eye(A.shape[1])) @ A.T
# def calc_proj_sparse(R, w,reg=1e-10):
#     """ Returns projection of R(ates) onto the space defined by w """
#     P = calc_proj_matrix_sparse(w,reg=reg)
#     return P@R.T


In [21]:
N = 10
X = act_trial_mean_activity

y=np.array(cos_x).reshape((n_trials,-1,1))[:,0,:]
X_proc = X
axes_list_x = nans([N,y.shape[1],dim])
r2_cv_list_x = nans([N])
r2_list_x = nans([N])
for i in range(N):
    reg = GridSearchCV(Ridge(), {'alpha': np.logspace(-3, 3, 7)}).fit(X_proc, y)
    # reg = GridSearchCV(Lasso(), {'alpha': np.logspace(-3, 3, 7)}).fit(X_proc, y)
    axes_list_x[i,:,:] = reg.best_estimator_.coef_
    r2_list_x[i] = reg.best_estimator_.score(X_proc, y)
    weights = reg.best_estimator_.coef_
    skf = StratifiedKFold(n_splits=5,shuffle=True,random_state = 42)   
    true_concat = nans([n_trials,1])
    pred_concat = nans([n_trials,1])
    trial_save_idx = 0
    for training_set, test_set in skf.split(range(0,n_trials),cond_dict):
        #split training and testing by trials
        X_train, X_test = X_proc[training_set,:],X_proc[test_set,:]
        y_train, y_test = y[training_set],y[test_set]
        lr = GridSearchCV(Ridge(), {'alpha': np.logspace(-3, 3, 7)})
        # lr = GridSearchCV(Lasso(), {'alpha': np.logspace(-3, 3, 7)})
        lr.fit(X_train, y_train)
        y_test_predicted = lr.predict(X_test)

        n = y_test_predicted.shape[0]
        true_concat[trial_save_idx:trial_save_idx+n] = y_test
        pred_concat[trial_save_idx:trial_save_idx+n] = y_test_predicted.reshape(len(y_test_predicted),1)
        trial_save_idx += n
    sses =get_sses_pred(true_concat,pred_concat)
    sses_mean=get_sses_mean(true_concat)
    R2 =1-np.sum(sses)/np.sum(sses_mean)   
    r2_cv_list_x[i] = R2
    X_proc = X_proc - calc_proj(X_proc,weights.T).T
    # X_proc = X_proc - calc_proj_sparse(X_proc,weights.T).T

X_proc = X
y = np.array(sin_y).reshape((n_trials,-1,1))[:,0,:]
axes_list_y = nans([N,y.shape[1],dim])
r2_cv_list_y = nans([N])
r2_list_y = nans([N])
for i in range(N):
    reg = GridSearchCV(Ridge(), {'alpha': np.logspace(-3, 3, 7)}).fit(X_proc, y)
    # reg = GridSearchCV(Lasso(), {'alpha': np.logspace(-3, 3, 7)}).fit(X_proc, y)
    axes_list_y[i,:,:] = reg.best_estimator_.coef_
    r2_list_y[i] = reg.best_estimator_.score(X_proc, y)
    weights = reg.best_estimator_.coef_
    skf = StratifiedKFold(n_splits=5,shuffle=True,random_state = 42)   
    true_concat = nans([n_trials,1])
    pred_concat = nans([n_trials,1])
    trial_save_idx = 0
    for training_set, test_set in skf.split(range(0,n_trials),cond_dict):
        #split training and testing by trials
        X_train, X_test = X_proc[training_set,:],X_proc[test_set,:]
        y_train, y_test = y[training_set],y[test_set]
        lr = GridSearchCV(Ridge(), {'alpha': np.logspace(-3, 3, 7)})
        # lr = GridSearchCV(Lasso(), {'alpha': np.logspace(-3, 3, 7)})
        lr.fit(X_train, y_train)
        y_test_predicted = lr.predict(X_test)

        n = y_test_predicted.shape[0]
        true_concat[trial_save_idx:trial_save_idx+n] = y_test
        pred_concat[trial_save_idx:trial_save_idx+n] = y_test_predicted.reshape(len(y_test_predicted),1)
        trial_save_idx += n
    sses =get_sses_pred(true_concat,pred_concat)
    sses_mean=get_sses_mean(true_concat)
    R2 =1-np.sum(sses)/np.sum(sses_mean)   
    r2_cv_list_y[i] = R2
    X_proc = X_proc - calc_proj(X_proc,weights.T).T
    # X_proc = X_proc - calc_proj_sparse(X_proc,weights.T).T

In [ ]:
plt.plot(np.arange(N)+1,r2_cv_list_x)
plt.xlabel('N')
plt.title('x-dir')
plt.ylabel('R2')
print(r2_cv_list_x)
plt.show()
plt.plot(np.arange(N)+1,r2_cv_list_y)
plt.xlabel('N')
plt.title('y-dir')
plt.ylabel('R2')
print(r2_cv_list_y)

In [ ]:
thresh = 0.0
print(axes_list_x.squeeze()[r2_cv_list_x>thresh,:].shape)
print(axes_list_y.squeeze()[r2_cv_list_y>thresh,:].shape)
CD_axes = np.vstack((axes_list_x.squeeze()[r2_cv_list_x>thresh,:],axes_list_y.squeeze()[r2_cv_list_y>thresh,:]))
CD_axes.shape

all_data = dataset_10ms.data[x_field].to_numpy()
proj_data = all_data @ CD_axes.T
print(proj_data.shape)

dataset_10ms.add_continuous_data(proj_data,'unsmoothed100_CD_proj_'+x_field)

# reg = 'Lasso'
# dataset_10ms.add_continuous_data(proj_data,'CD_proj_'+x_field+reg)

In [ ]:
## Find FB axes
proj_out_CD = True
align_range = (200, 400)
align_field = 'move_onset_time'

mask = active_mask
active_df = dataset.make_trial_data(align_field=align_field, align_range=align_range, ignored_trials=~mask)
cond_dict = active_cond_dict
n_trials = active_df['trial_id'].nunique()
X = active_df[x_field].to_numpy().reshape((-1, dim))
# X = (X - mean)/std
act_trial_spikes = X.reshape((n_trials, -1, dim))
print(act_trial_spikes.shape)
act_trial_mean_activity = np.mean(act_trial_spikes, axis=1)

dirs = [dataset.trial_info[dataset.trial_info.trial_id == i]['cond_dir'] for i in active_df.trial_id]
cos_x = [round(math.cos(math.radians(i)),3) for i in dirs]
sin_y = [round(math.sin(math.radians(i)),3) for i in dirs]
cos_sin = np.array([cos_x, sin_y]).T
print(cos_sin.shape)

# dirs = [dataset.trial_info[dataset.trial_info.trial_id == i]['cond_dir'] for i in active_df.trial_id]
# filter = np.array([x%90==0 for x in dirs])
# n_trials = len(filter)
# act_trial_mean_activity = act_trial_mean_activity[filter[0::act_trial_spikes.shape[1]].squeeze(),:]
# print(act_trial_mean_activity.shape)

# cos_x = np.array([round(math.cos(math.radians(i)),3) for i in dirs])[filter.squeeze()]
# sin_y = np.array([round(math.sin(math.radians(i)),3) for i in dirs])[filter.squeeze()]
# cos_sin = np.array([cos_x, sin_y]).T
# print(cos_sin.shape)

# n_trials = act_trial_mean_activity.shape[0]
# cond_dict = cond_dict[filter[0::act_trial_spikes.shape[1]].squeeze()]

In [25]:
N = 10
X = act_trial_mean_activity
if proj_out_CD:
    X =  X - calc_proj(X,CD_axes.T).T

y=np.array(cos_x).reshape((n_trials,-1,1))[:,0,:]
X_proc = X
axes_list_x = nans([N,y.shape[1],dim])
r2_cv_list_x = nans([N])
r2_list_x = nans([N])
for i in range(N):
    reg = GridSearchCV(Ridge(), {'alpha': np.logspace(-3, 3, 7)}).fit(X_proc, y)
    # reg = GridSearchCV(Lasso(), {'alpha': np.logspace(-3, 3, 7)}).fit(X_proc, y)
    axes_list_x[i,:,:] = reg.best_estimator_.coef_
    r2_list_x[i] = reg.best_estimator_.score(X_proc, y)
    weights = reg.best_estimator_.coef_
    kf = KFold(n_splits=3,shuffle=True,random_state = 42)   
    true_concat = nans([n_trials,1])
    pred_concat = nans([n_trials,1])
    trial_save_idx = 0
    for training_set, test_set in kf.split(range(0,n_trials)):
        #split training and testing by trials
        X_train, X_test = X_proc[training_set,:],X_proc[test_set,:]
        y_train, y_test = y[training_set],y[test_set]
        lr = GridSearchCV(Ridge(), {'alpha': np.logspace(-3, 3, 7)})
        # lr = GridSearchCV(Lasso(), {'alpha': np.logspace(-3, 3, 7)})
        lr.fit(X_train, y_train)
        y_test_predicted = lr.predict(X_test)

        n = y_test_predicted.shape[0]
        true_concat[trial_save_idx:trial_save_idx+n] = y_test
        pred_concat[trial_save_idx:trial_save_idx+n] = y_test_predicted.reshape(len(y_test_predicted),1)
        trial_save_idx += n
    sses =get_sses_pred(true_concat,pred_concat)
    sses_mean=get_sses_mean(true_concat)
    R2 =1-np.sum(sses)/np.sum(sses_mean)   
    r2_cv_list_x[i] = R2
    X_proc = X_proc - calc_proj(X_proc,weights.T).T

X_proc = X
y = np.array(sin_y).reshape((n_trials,-1,1))[:,0,:]
axes_list_y = nans([N,y.shape[1],dim])
r2_cv_list_y = nans([N])
r2_list_y = nans([N])
for i in range(N):
    reg = GridSearchCV(Ridge(), {'alpha': np.logspace(-3, 3, 7)}).fit(X_proc, y)
    # reg = GridSearchCV(Lasso(), {'alpha': np.logspace(-3, 3, 7)}).fit(X_proc, y)
    axes_list_y[i,:,:] = reg.best_estimator_.coef_
    r2_list_y[i] = reg.best_estimator_.score(X_proc, y)
    weights = reg.best_estimator_.coef_
    kf = KFold(n_splits=3,shuffle=True,random_state = 42)   
    true_concat = nans([n_trials,1])
    pred_concat = nans([n_trials,1])
    trial_save_idx = 0
    for training_set, test_set in kf.split(range(0,n_trials)):
        #split training and testing by trials
        X_train, X_test = X_proc[training_set,:],X_proc[test_set,:]
        y_train, y_test = y[training_set],y[test_set]
        lr = GridSearchCV(Ridge(), {'alpha': np.logspace(-3, 3, 7)})
        # lr = GridSearchCV(Lasso(), {'alpha': np.logspace(-3, 3, 7)})
        lr.fit(X_train, y_train)
        y_test_predicted = lr.predict(X_test)

        n = y_test_predicted.shape[0]
        true_concat[trial_save_idx:trial_save_idx+n] = y_test
        pred_concat[trial_save_idx:trial_save_idx+n] = y_test_predicted.reshape(len(y_test_predicted),1)
        trial_save_idx += n
    sses =get_sses_pred(true_concat,pred_concat)
    sses_mean=get_sses_mean(true_concat)
    R2 =1-np.sum(sses)/np.sum(sses_mean)   
    r2_cv_list_y[i] = R2
    X_proc = X_proc - calc_proj(X_proc,weights.T).T

In [ ]:
plt.plot(np.arange(N)+1,r2_cv_list_x)
plt.xlabel('N')
plt.title('x-dir')
plt.ylabel('R2')
print(r2_cv_list_x)
plt.show()
plt.plot(np.arange(N)+1,r2_cv_list_y)
plt.xlabel('N')
plt.title('y-dir')
plt.ylabel('R2')
print(r2_cv_list_y)

In [ ]:
thresh = 0.0
print(axes_list_x.squeeze()[r2_cv_list_x>thresh,:].shape)
print(axes_list_y.squeeze()[r2_cv_list_y>thresh,:].shape)
FB_axes = np.vstack((axes_list_x.squeeze()[r2_cv_list_x>thresh,:],axes_list_y.squeeze()[r2_cv_list_y>thresh,:]))
FB_axes.shape

# # thresh = 0.08
# # print(axes_list_x.squeeze()[r2_list_x>thresh,:].shape)
# # print(axes_list_y.squeeze()[r2_list_y>thresh,:].shape)
# # FB_axes_alt = np.vstack((axes_list_x.squeeze()[r2_list_x>thresh,:],axes_list_y.squeeze()[r2_list_y>thresh,:]))
# # FB_axes_alt.shape

all_data = dataset_10ms.data[x_field].to_numpy()
proj_data = all_data @ FB_axes.T
print(proj_data.shape)

dataset_10ms.add_continuous_data(proj_data,'unsmoothed100_FB_proj_'+x_field)

# reg='Lasso'
# dataset_10ms.add_continuous_data(proj_data,'FB_proj_'+x_field+reg)

In [ ]:
print(dataset.data.keys().unique(0))
CD_proj = np.array(dataset_10ms.data['unsmoothed100_CD_proj_'+x_field])
print(CD_proj.shape)
FB_proj = np.array(dataset_10ms.data['unsmoothed100_FB_proj_'+x_field])
print(FB_proj.shape)
CD_FB_proj  = np.hstack([CD_proj,FB_proj])
print(CD_FB_proj.shape)
dataset_10ms.add_continuous_data(CD_FB_proj,'unsmoothed100_CD_FB_proj_'+x_field)

# # all_data = dataset_10ms.data[x_field].to_numpy()
# # CD_proj_out_data = all_data - calc_proj(all_data,CD_axes.T).T
# # print(CD_proj_out_data.shape)
# # dataset_10ms.add_continuous_data(CD_proj_out_data,'CD_proj_out_'+x_field)

In [ ]:
# CD_axes_Lasso = CD_axes
# FB_axes_Lasso = FB_axes

# CD_axes_Ridge = CD_axes
# FB_axes_Ridge = FB_axes

In [29]:
np.savez(monkey+'_unsmoothed100_cdfb_weights_'+x_field, CD_axes = CD_axes, FB_axes = FB_axes) 
np.savez(monkey+'_unsmoothed100_cdfb_data_'+x_field, \
        CD_FB_proj = dataset.data['unsmoothed100_CD_FB_proj_'+x_field].to_numpy(), \
        FB_proj = dataset.data['unsmoothed100_FB_proj_'+x_field].to_numpy(),
        CD_proj = dataset.data['unsmoothed100_CD_proj_'+x_field].to_numpy())

# np.savez(monkey+'_cdfb_weights_'+x_field, CD_axes_Lasso = CD_axes_Lasso, FB_axes_Lasso = FB_axes_Lasso,\
#          CD_axes_Ridge = CD_axes_Ridge, FB_axes_Ridge = FB_axes_Ridge) 
# np.savez(monkey+'_cdfb_data_'+x_field, \
#         CD_FB_proj_Lasso = dataset.data['CD_FB_proj_'+x_field+'Lasso'].to_numpy(), \
#         FB_proj_Lasso = dataset.data['FB_proj_'+x_field+'Lasso'].to_numpy(),\
#         CD_proj_Lasso = dataset.data['CD_proj_'+x_field+'Lasso'].to_numpy(),\
#         CD_FB_proj_Ridge = dataset.data['CD_FB_proj_'+x_field+'Ridge'].to_numpy(), \
#         FB_proj_Ridge = dataset.data['FB_proj_'+x_field+'Ridge'].to_numpy(),
#         CD_proj_Ridge = dataset.data['CD_proj_'+x_field+'Ridge'].to_numpy())

## Angles

In [ ]:
X = CD_axes.T
Y = FB_axes.T
angles = principal_angles(X, Y)
print("Principal angles", np.degrees(angles))

In [ ]:
FB_axes_plot = FB_axes
angDist_array = nans([len(CD_axes),len(FB_axes_plot)])
for i in range(len(CD_axes)):
    for j in range(len(FB_axes_plot)):
        angDist_array[i,j] = math.degrees(angle_between(CD_axes[i,:],FB_axes_plot[j,:]))
fig, ax = plt.subplots(figsize=(4, 4))
im = ax.imshow(angDist_array)
ax.set_xlabel('Feedback axes')
ax.set_ylabel('CD axes')
ax.set_xticks(np.arange(len(FB_axes_plot)))
ax.set_yticks(np.arange(len(CD_axes)))

for i in range(len(CD_axes)):
    for j in range(len(FB_axes_plot)):
        text = ax.text(j, i, str(int(angDist_array[i, j])),
                        ha="center", va="center", color="w", fontsize=14)
plt.tight_layout()
# plt.savefig(figDir + monkey + '_cdfb_degrees_pc.pdf',dpi = 'figure')

In [ ]:
# FB_axes_plot = FB_axes
# angDist_array = nans([len(FB_axes_plot),len(FB_axes_plot)])
# for i in range(len(FB_axes_plot)):
#     for j in range(len(FB_axes_plot)):
#         angDist_array[i,j] = math.degrees(angle_between(FB_axes_plot[i,:],FB_axes_plot[j,:]))
# fig, ax = plt.subplots(figsize=(6, 6))
# im = ax.imshow(angDist_array)
# ax.set_xlabel('FB axes')
# ax.set_ylabel('FB axes')
# ax.set_xticks(np.arange(len(FB_axes_plot)))
# ax.set_yticks(np.arange(len(FB_axes_plot)))
# for i in range(len(FB_axes_plot)):
#     for j in range(len(FB_axes_plot)):
#         text = ax.text(j, i, str(int(angDist_array[i, j])),
#                         ha="center", va="center", color="w", fontsize=14)
# plt.tight_layout()

# angDist_array = nans([len(CD_axes),len(CD_axes)])
# for i in range(len(CD_axes)):
#     for j in range(len(CD_axes)):
#         angDist_array[i,j] = math.degrees(angle_between(CD_axes[i,:],CD_axes[j,:]))
# fig, ax = plt.subplots(figsize=(4, 4))
# im = ax.imshow(angDist_array)
# ax.set_xlabel('CD axes')
# ax.set_ylabel('CD axes')
# ax.set_xticks(np.arange(len(CD_axes)))
# ax.set_yticks(np.arange(len(CD_axes)))
# for i in range(len(CD_axes)):
#     for j in range(len(CD_axes)):
#         text = ax.text(j, i, str(int(angDist_array[i, j])),
#                         ha="center", va="center", color="w", fontsize=14)
# plt.tight_layout()

## Plot

In [ ]:
# Plot PCA projections over trial, for different reaching directions
plot_dir = np.array([0,45,90,135,180,225,270,315]) 
directions = np.array([0,45,90,135,180,225,270,315])
# plot_dir = np.array([0,90,180,270]) 
# directions = np.array([0,90,180,270])
cmap = plt.get_cmap('coolwarm',len(plot_dir))
custom_palette = [mpl.colors.rgb2hex(cmap(i)) for i in range(len(plot_dir))]
plot_field = 'unsmoothed100_CD_FB_proj_'+x_field
N = dataset.data[plot_field].shape[1]
order = range(N)

pred_range = (-100, 1100)
trial_mask = active_mask
cond_dict = active_cond_dict
n_timepoints = int((pred_range[1] - pred_range[0])/dataset_10ms.bin_width)
data = dataset_10ms.make_trial_data(align_field='move_onset_time', align_range=pred_range, ignored_trials=~trial_mask, allow_overlap=True)
n_trials = data['trial_id'].nunique()
trials_pca = nans([n_trials,n_timepoints,N])
i = 0
for idx, trial in data.groupby('trial_id'):
    trials_pca[i,:,:]=trial[plot_field].to_numpy()
    i+=1
print(trials_pca.shape)

x_axis = np.arange(pred_range[0], pred_range[1], dataset_10ms.bin_width)

# define some useful time points
move_idx=0
ret_idx = 500

plot_dims = N

fig,ax=plt.subplots(plot_dims,1,figsize=(10,N+4))
for i in range(plot_dims):
    for j in range(len(plot_dir)):
        color = custom_palette[j]
        dir_idx = np.argwhere(directions == plot_dir[j])[0]
        cond_mean_proj = np.mean(trials_pca[np.argwhere(cond_dict==dir_idx).flatten(),:,:], axis = 0)[:,order[i]] 
        pca_mean = np.mean(data[plot_field].to_numpy(),axis = 0)[order[i]] 
        ax[i].plot(x_axis,cond_mean_proj - pca_mean,linewidth=2.25,color = color,label = plot_dir[j])
        
        ax[i].axvline(move_idx, color='k',linewidth = .5)
        ax[i].axvline(ret_idx, color='k',linewidth = .5)
        ax[i].set_xlim([-100,1000])
        # ax[i].set_ylim([-15, 15])
        ax[i].axhline(0,color ='k',ls = '--')
        if i<plot_dims-1:
            ax[i].set_xticks([])
        else:
            ax[i].set_xlabel('Time after movement onset (ms)')
            
        ax[i].set_yticks([])
        ax[i].set_ylabel('Dim. '+str(i+1))

    ax[0].set_title('Active trials ' + plot_field)
     
plt.legend(bbox_to_anchor = (1, 1), loc = 'upper left')
plt.tight_layout()
# plt.savefig(figDir + monkey + '_cdfb_active_pc.pdf',dpi = 'figure')

In [ ]:
# Plot PCA projections over trial, for different reaching directions
pred_range = (-100, 600)
trial_mask = passive_mask
cond_dict = passive_cond_dict
n_timepoints = int((pred_range[1] - pred_range[0])/dataset_10ms.bin_width)
data = dataset_10ms.make_trial_data(align_field='move_onset_time', align_range=pred_range, ignored_trials=~trial_mask, allow_overlap=True)
n_trials = data['trial_id'].nunique()
trials_pca = nans([n_trials,n_timepoints,N])
i = 0
for idx, trial in data.groupby('trial_id'):
    trials_pca[i,:,:]=trial[plot_field].to_numpy()
    i+=1
print(trials_pca.shape)

x_axis = np.arange(pred_range[0], pred_range[1], dataset_10ms.bin_width)

# define some useful time points
move_idx=0
ret_idx = 200

plot_dims = N

fig,ax=plt.subplots(plot_dims,1,figsize=(10,N+4))
for i in range(plot_dims):
    for j in range(len(plot_dir)):
        color = custom_palette[j]
        dir_idx = np.argwhere(directions == plot_dir[j])[0]
        cond_mean_proj = np.mean(trials_pca[np.argwhere(cond_dict==dir_idx).flatten(),:,:], axis = 0)[:,order[i]] 
        pca_mean = np.mean(data[plot_field].to_numpy(),axis = 0)[order[i]]
        ax[i].plot(x_axis,cond_mean_proj - pca_mean,linewidth=2.25,color = color,label = plot_dir[j])
        
        ax[i].axvline(move_idx, color='k',linewidth = .5)
        ax[i].axvline(ret_idx, color='k',linewidth = .5)
        ax[i].set_xlim([-100,500])
        # ax[i].set_ylim([-15, 15])
        ax[i].axhline(0,color ='k',ls = '--')
        if i<plot_dims-1:
            ax[i].set_xticks([])
        else:
            ax[i].set_xlabel('Time after movement onset (ms)')
            
        ax[i].set_yticks([])
        ax[i].set_ylabel('Dim. '+str(i+1))

    ax[0].set_title('Passive trials ' + plot_field)
    
plt.legend(bbox_to_anchor = (1, 1), loc = 'upper left')
plt.tight_layout()
# plt.savefig(figDir + monkey + '_cdfb_passive_pc.pdf',dpi = 'figure')

In [ ]:
# Plot PCA projections over trial, for different reaching directions
pred_range = (-100, 1100)
trial_mask = nan_mask
# cond_dict = nan_bump_cond_dict
cond_dict = nan_cond_dict
n_timepoints = int((pred_range[1] - pred_range[0])/dataset_10ms.bin_width)
data = dataset_10ms.make_trial_data(align_field='move_onset_time', align_range=pred_range, ignored_trials=~trial_mask, allow_overlap=True)
n_trials = data['trial_id'].nunique()
trials_pca = nans([n_trials,n_timepoints,N])
i = 0
for idx, trial in data.groupby('trial_id'):
    trials_pca[i,:,:]=trial[plot_field].to_numpy()
    i+=1
print(trials_pca.shape)

x_axis = np.arange(pred_range[0], pred_range[1], dataset_10ms.bin_width)

# define some useful time points
move_idx=0
ret_idx = 200

plot_dims = N

fig,ax=plt.subplots(plot_dims,1,figsize=(10,N+4))
for i in range(plot_dims):
    for j in range(len(plot_dir)):
        color = custom_palette[j]
        dir_idx = np.argwhere(directions == plot_dir[j])[0]
        cond_mean_proj = np.mean(trials_pca[np.argwhere(cond_dict==dir_idx).flatten(),:,:], axis = 0)[:,order[i]] 
        pca_mean = np.mean(data[plot_field].to_numpy(),axis = 0)[order[i]]
        ax[i].plot(x_axis,cond_mean_proj - pca_mean,linewidth=2.25,color = color,label = plot_dir[j])
        
        ax[i].axvline(move_idx, color='k',linewidth = .5)
        ax[i].axvline(ret_idx, color='k',linewidth = .5)
        ax[i].set_xlim([-100,1000])
        # ax[i].set_ylim([-15, 15])
        ax[i].axhline(0,color ='k',ls = '--')
        if i<plot_dims-1:
            ax[i].set_xticks([])
        else:
            ax[i].set_xlabel('Time after movement onset (ms)')
            
        ax[i].set_yticks([])
        ax[i].set_ylabel('Dim. '+str(i+1))

    ax[0].set_title('Active-bump trials ' + plot_field)
    
plt.legend(bbox_to_anchor = (1, 1), loc = 'upper left')
plt.tight_layout()
# plt.savefig(figDir + monkey + '_cdfb_passive_pc.pdf',dpi = 'figure')

In [ ]:
plot_field = 'CD_proj_'+x_field
x_name = '0000'
y_name = '0001'

# Active, 2D plot
unique_conditions = [(False, 0.0), (False, 45.0), (False, 90.0), (False, 135.0),
                     (False, 180.0), (False, 225.0), (False, 270.0), (False, 315.0)]
# unique_conditions = [(False, 0.0),  (False, 90.0), (False, 180.0), (False, 270.0)]

# Initialize figure
fig = plt.figure(figsize=(20, 4))
ax_0 = fig.add_subplot(1,5,1)
ax_1 = fig.add_subplot(1, 5,2)
ax_2 = fig.add_subplot(1, 5,3)
ax_3 = fig.add_subplot(1, 5,4)
ax_4 = fig.add_subplot(1, 5,5)
# xlim = [-1.5, 1.5]
# ylim = [-1.5, 1.5]

for cond in unique_conditions:
    # Filter out invalid trials (labeled 'none') and trials in other conditions
    cond_mask = (dataset.trial_info['ctr_hold_bump']==cond[0]) & \
                (dataset.trial_info['cond_dir']%360==cond[1]) & \
                (dataset.trial_info.split != 'none')
# cond_mask = (np.isnan(dataset.trial_info['ctr_hold_bump'])) & \

    # Extract relevant portion of selected trials
    cond_data = dataset.make_trial_data(align_field='move_onset_time', align_range=(-200, 0), ignored_trials=~cond_mask)
    ax_0.plot(cond_data.groupby('align_time').mean()[plot_field][x_name].to_numpy(),cond_data.groupby('align_time').mean()[plot_field][y_name].to_numpy(),color=plt.cm.hsv(cond[1] / 360))
    # for idx, trial in cond_data.groupby('trial_id'):
    #     ax_0.plot(trial[plot_field][x_name], trial[plot_field][y_name], color=plt.cm.hsv(cond[1] / 360), linewidth=0.5)
        # ax_0.set_xlim(xlim)
        # ax_0.set_ylim(ylim)

    cond_data = dataset.make_trial_data(align_field='move_onset_time', align_range=(0, 500), ignored_trials=~cond_mask)
    ax_1.plot(cond_data.groupby('align_time').mean()[plot_field][x_name].to_numpy(),cond_data.groupby('align_time').mean()[plot_field][y_name].to_numpy(),color=plt.cm.hsv(cond[1] / 360))
    # for idx, trial in cond_data.groupby('trial_id'):
    #     ax_1.plot(trial[plot_field][x_name], trial[plot_field][y_name], color=plt.cm.hsv(cond[1] / 360), linewidth=0.5)
        # ax_1.set_xlim(xlim)
        # ax_1.set_ylim(ylim)
    
    cond_data = dataset.make_trial_data(align_field='move_onset_time', align_range=(500, 1000), ignored_trials=~cond_mask)
    ax_2.plot(cond_data.groupby('align_time').mean()[plot_field][x_name].to_numpy(),cond_data.groupby('align_time').mean()[plot_field][y_name].to_numpy(),color=plt.cm.hsv(cond[1] / 360))
    # for idx, trial in cond_data.groupby('trial_id'):
    #     ax_2.plot(trial[plot_field][x_name], trial[plot_field][y_name], color=plt.cm.hsv(cond[1] / 360), linewidth=0.5)
        # ax_2.set_xlim(xlim)
        # ax_2.set_ylim(ylim)
            
    cond_data = dataset.make_trial_data(align_field='move_onset_time', align_range=(1000, 1500), ignored_trials=~cond_mask)
    ax_3.plot(cond_data.groupby('align_time').mean()[plot_field][x_name].to_numpy(),cond_data.groupby('align_time').mean()[plot_field][y_name].to_numpy(),color=plt.cm.hsv(cond[1] / 360))
    # for idx, trial in cond_data.groupby('trial_id'):
    #     ax_3.plot(trial[plot_field][x_name], trial[plot_field][y_name], color=plt.cm.hsv(cond[1] / 360), linewidth=0.5)
        # ax_3.set_xlim(xlim)
        # ax_3.set_ylim(ylim)

    cond_data = dataset.make_trial_data(align_field='move_onset_time', align_range=(1500, 2000), ignored_trials=~cond_mask)
    ax_4.plot(cond_data.groupby('align_time').mean()[plot_field][x_name].to_numpy(),cond_data.groupby('align_time').mean()[plot_field][y_name].to_numpy(),color=plt.cm.hsv(cond[1] / 360))
    # for idx, trial in cond_data.groupby('trial_id'):
    #     ax_4.plot(trial[plot_field][x_name], trial[plot_field][y_name], color=plt.cm.hsv(cond[1] / 360), linewidth=0.5)
        # ax_4.set_xlim(xlim)
        # ax_4.set_ylim(ylim)
            
# Add labels
ax_0.set_title('-200 to 0')
ax_1.set_title('0 to 500')
ax_2.set_title('500 to 1000')
ax_3.set_title('1000 to 1500')
ax_4.set_title('1500 to 2000')

# ax_0.axis("off")
# ax_1.axis("off")
# ax_2.axis("off")
# ax_3.axis("off")
# ax_4.axis("off")

plt.tight_layout()
# plt.savefig(figDir + monkey + '_active_traj.pdf',dpi = 'figure')
# plt.suptitle('Active Reach Trajectories')
plt.show()

In [ ]:
# Passive

unique_conditions = [(True, 0.0), (True, 45.0), (True, 90.0), (True, 135.0),
                     (True, 180.0), (True, 225.0), (True, 270.0), (True, 315.0)]
# unique_conditions = [(True, 0.0),  (True, 90.0), (True, 180.0), (True, 270.0)]

# Initialize figure
fig = plt.figure(figsize=(12, 4))
ax_0 = fig.add_subplot(1,3,1)
ax_1 = fig.add_subplot(1, 3,2)
ax_2 = fig.add_subplot(1, 3,3)
# xlim = [-1.5, 1.5]
# ylim = [-1.5, 1.5]

for cond in unique_conditions:
    # Filter out invalid trials (labeled 'none') and trials in other conditions
    cond_mask = (dataset.trial_info['ctr_hold_bump']==cond[0]) & \
                (dataset.trial_info['cond_dir']%360==cond[1]) & \
                (dataset.trial_info.split != 'none')
# cond_mask = (np.isnan(dataset.trial_info['ctr_hold_bump'])) & \

    # Extract relevant portion of selected trials
    cond_data = dataset.make_trial_data(align_field='move_onset_time', align_range=(-200, 0), ignored_trials=~cond_mask)
    ax_0.plot(cond_data.groupby('align_time').mean()[plot_field][x_name].to_numpy(),cond_data.groupby('align_time').mean()[plot_field][y_name].to_numpy(),color=plt.cm.hsv(cond[1] / 360))
    # for idx, trial in cond_data.groupby('trial_id'):
    #     ax_0.plot(trial[plot_field][x_name], trial[plot_field][y_name], color=plt.cm.hsv(cond[1] / 360), linewidth=0.5)
        # ax_0.set_xlim(xlim)
        # ax_0.set_ylim(ylim)
    
    cond_data = dataset.make_trial_data(align_field='move_onset_time', align_range=(0, 200), ignored_trials=~cond_mask)
    ax_1.plot(cond_data.groupby('align_time').mean()[plot_field][x_name].to_numpy(),cond_data.groupby('align_time').mean()[plot_field][y_name].to_numpy(),color=plt.cm.hsv(cond[1] / 360))
    # for idx, trial in cond_data.groupby('trial_id'):
    #     ax_1.plot(trial[plot_field][x_name], trial[plot_field][y_name], color=plt.cm.hsv(cond[1] / 360), linewidth=0.5)
        # ax_1.set_xlim(xlim)
        # ax_1.set_ylim(ylim)

    cond_data = dataset.make_trial_data(align_field='move_onset_time', align_range=(200, 500), ignored_trials=~cond_mask)
    ax_2.plot(cond_data.groupby('align_time').mean()[plot_field][x_name].to_numpy(),cond_data.groupby('align_time').mean()[plot_field][y_name].to_numpy(),color=plt.cm.hsv(cond[1] / 360))
    # for idx, trial in cond_data.groupby('trial_id'):
    #     ax_2.plot(trial[plot_field][x_name], trial[plot_field][y_name], color=plt.cm.hsv(cond[1] / 360), linewidth=0.5)
        # ax_2.set_xlim(xlim)
        # ax_2.set_ylim(ylim)
            
# Add labels
ax_0.set_title('-200 to 0')
ax_1.set_title('0 to 200')
ax_2.set_title('200 to 500')

# ax_0.axis("off")
# ax_1.axis("off")
# ax_2.axis("off")
# ax_3.axis("off")
# ax_4.axis("off")

figDir = '/Users/sherryan/area2_population_analysis/figures_plus/'
plt.tight_layout()
# plt.savefig(figDir + monkey + '_active_traj.pdf',dpi = 'figure')
# plt.suptitle('Active Reach Trajectories')
plt.show()

# Reaching directions

In [26]:
foldername = "~/area2_population_analysis/s1-kinematics/actpas_NWB/"
monkey = "Han_20171207"
filename = foldername + monkey + "_COactpas_TD.nwb"

# monkey = "Chips_20170913"
# filename = foldername + monkey + "_COactpas_TD.nwb"

# monkey = 'Duncan_20190710'
# filename = foldername + monkey + "_COactpas_reformat.nwb"

dataset_50ms = NWBDataset(filename, split_heldout=False)
# dataset_50ms.resample(50)
# bin_width = dataset_50ms.bin_width
# print(bin_width)

dataset_50ms.resample(10)
bin_width = dataset_50ms.bin_width
print(bin_width)

# filename = '/Users/sherryan/area2_population_analysis/s1-kinematics/'+monkey+'_COactpas_with_emg_TD.mat'
# mat = scipy.io.loadmat(filename)
# EMG = mat['trial_data']['emg'][0,0]
# dataset_50ms.add_continuous_data(EMG,'EMG')

# dataset_50ms.add_continuous_data(dataset_10ms.data.PCA_40.to_numpy(),'PCA_40')
# dataset_50ms.add_continuous_data(dataset_10ms.data.spikes_smth_40_oneside.to_numpy(),'spikes_smth_40_oneside')
# dataset_50ms.add_continuous_data(dataset_10ms.data.muscle_PCA.to_numpy(),'muscle_PCA')
# dataset_50ms.add_continuous_data(dataset_10ms.data.joint_PCA.to_numpy(),'joint_PCA')

x_field = 'spikes_smth_100_oneside'
data = np.load(monkey+'_ac100_cdfb_data_'+x_field+'.npz')
data.files
dataset_50ms.add_continuous_data(data['CD_FB_proj'],'CD_FB_proj')
dataset_50ms.add_continuous_data(data['CD_proj'],'CD_proj')
dataset_50ms.add_continuous_data(data['FB_proj'],'FB_proj')
dataset_50ms.resample(50)

In [27]:
def procX_train_test(X,y,training_set,test_set):
    X_train = X[training_set,:]
    X_test = X[test_set,:]
    y_train = y[training_set,:]
    y_test = y[test_set,:]    
    
    X_train_mean=np.nanmean(X_train,axis=0)
    X_train_std=np.nanstd(X_train,axis=0)   

    X_train=(X_train-X_train_mean)/X_train_std
    X_test=(X_test-X_train_mean)/X_train_std
 
    return X_train,X_test,y_train,y_test

In [ ]:
#make dictionary for trial condition (reaching directions)
dataset = dataset_50ms
active_mask = (dataset.trial_info.ctr_hold_bump==0) & (dataset.trial_info['split'] != 'none')
passive_mask = (dataset.trial_info.ctr_hold_bump==1) & (dataset.trial_info['split'] != 'none')
nan_mask = (np.isnan(dataset.trial_info.ctr_hold_bump)) & (dataset.trial_info['split'] != 'none')
all_mask = (dataset.trial_info['split'] != 'none')

trial_mask = all_mask
valid_n_trials = dataset.trial_info.loc[trial_mask].shape[0]
print(valid_n_trials,'valid trials')


trial_mask = active_mask
active_trials_idx = np.array(dataset.trial_info.loc[trial_mask]['trial_id'])
active_n_trials = dataset.trial_info.loc[trial_mask].shape[0]
print(active_n_trials,'active trials')

trial_mask = passive_mask
passive_trials_idx = np.array(dataset.trial_info.loc[trial_mask]['trial_id'])
passive_n_trials = dataset.trial_info.loc[trial_mask].shape[0]
print(passive_n_trials,'passive trials')

trial_mask = nan_mask
nan_trials_idx = np.array(dataset.trial_info.loc[trial_mask]['trial_id'])
nan_n_trials = dataset.trial_info.loc[trial_mask].shape[0]
print(nan_n_trials,'reach bump trials')

active_cond_dir_idx = []
passive_cond_dir_idx = []
nan_cond_dir_idx = []
nan_bump_cond_dir_idx = []
for direction in [0,45,90,135,180,225,270,315]:
# for direction in [0,90,180,270]:
    active_cond_dir_idx.append(np.where((dataset.trial_info['cond_dir']%360 == direction) & (dataset.trial_info['ctr_hold_bump'] == 0) & \
           (dataset.trial_info['split'] != 'none'))[0])
    passive_cond_dir_idx.append(np.where((dataset.trial_info['cond_dir']%360 == direction) & (dataset.trial_info['ctr_hold_bump'] == 1) & \
           (dataset.trial_info['split'] != 'none'))[0])
    nan_cond_dir_idx.append(np.where((dataset.trial_info['cond_dir']%360 == direction) & (np.isnan(dataset.trial_info.ctr_hold_bump)) & \
           (dataset.trial_info['split'] != 'none'))[0])
    nan_bump_cond_dir_idx.append(np.where((dataset.trial_info['bump_dir']%360 == direction) & (np.isnan(dataset.trial_info.ctr_hold_bump)) & \
           (dataset.trial_info['split'] != 'none'))[0])

active_cond_dict = nans([active_n_trials])
i = 0
for idx in active_trials_idx:
    for cond in range(0,len(active_cond_dir_idx)):
        if idx in active_cond_dir_idx[cond]:
            active_cond_dict[i] = cond
            break
    i+=1
print(active_cond_dict)
print(len(active_cond_dict))

passive_cond_dict = nans([passive_n_trials])
i = 0
for idx in passive_trials_idx:
    for cond in range(0,len(passive_cond_dir_idx)):
        if idx in passive_cond_dir_idx[cond]:
            passive_cond_dict[i] = cond
            break
    i+=1
print(passive_cond_dict)
print(len(passive_cond_dict))

nan_cond_dict = nans([nan_n_trials])
i = 0
for idx in nan_trials_idx:
    for cond in range(0,len(nan_cond_dir_idx)):
        if idx in nan_cond_dir_idx[cond]:
            nan_cond_dict[i] = cond
            break
    i+=1
print(nan_cond_dict)
print(len(nan_cond_dict))

nan_bump_cond_dict = nans([nan_n_trials])
i = 0
for idx in nan_trials_idx:
    for cond in range(0,len(nan_bump_cond_dir_idx)):
        if idx in nan_bump_cond_dir_idx[cond]:
            nan_bump_cond_dict[i] = cond
            break
    i+=1
print(nan_bump_cond_dict)
print(len(nan_bump_cond_dict))

In [ ]:
active_df = dataset_50ms.make_trial_data(align_field='move_onset_time', align_range = (-100,100), ignored_trials = ~active_mask)
del_indices = list(set(active_trials_idx) - set(active_df['trial_id'].unique()))
print('was',active_n_trials,'active trials')
active_n_trials = active_n_trials - len(list(set(active_trials_idx) - set(active_df['trial_id'].unique())))
active_cond_dict = np.delete(active_cond_dict,np.argwhere(active_trials_idx==del_indices)[0])
print('now',active_n_trials,'active trials')
print(len(active_cond_dict))

In [ ]:
n_dims = 20
active_mask = (dataset_50ms.trial_info.ctr_hold_bump==0) & (dataset_50ms.trial_info.split != 'none')
passive_mask = (dataset_50ms.trial_info.ctr_hold_bump==1) & (dataset_50ms.trial_info.split != 'none')

all_data = np.array(dataset_50ms.data.spikes)
print(all_data.shape)
if not np.isnan(all_data).any():
    scaler = StandardScaler()
    X = scaler.fit_transform(all_data)
    pca = PCA(n_components=n_dims,random_state = 42)
    PCA_data = pca.fit_transform(X)
print(PCA_data.shape)
dataset_50ms.add_continuous_data(PCA_data,'20PC')
print('PCA total var explained:',sum(pca.explained_variance_ratio_))

## Reach-bump trials

In [ ]:
pred_range = (-300, 1000)
x_field = 'CD_proj'
mask = nan_mask
n_trials = nan_n_trials
cond_dict = nan_cond_dict
dim = dataset.data[x_field].shape[1]
nan_df = dataset_50ms.make_trial_data(align_field='move_onset_time', align_range=pred_range, ignored_trials=~mask)
dirs = [dataset_50ms.trial_info[dataset_50ms.trial_info.trial_id == i]['cond_dir'] for i in nan_df.trial_id]
cos_x = [round(math.cos(math.radians(i)),3) for i in dirs]
sin_y = [round(math.sin(math.radians(i)),3) for i in dirs]
cos_sin = np.array([cos_x, sin_y]).T
nan_trial_PCA = nan_df[x_field].to_numpy().reshape((n_trials, -1, dim))
print(nan_trial_PCA.shape)
nan_trial_ang = cos_sin.reshape((n_trials, -1, 2))
# act_trial_ang = np.array(cos_x).reshape((n_trials,-1,1))
# act_trial_ang = np.array(sin_y).reshape((n_trials,-1,1))

print(nan_trial_ang.shape)

In [ ]:
nan_cond_r2_arr = nans([n_bins])
for i in range(n_bins):
    X = nan_trial_PCA[:,i,:]
    Y = nan_trial_ang[:,i,:]
    X_proc = (X - np.nanmean(X,axis=0))/np.nanstd(X,axis=0)
    lr_all = GridSearchCV(Ridge(), {'alpha': np.logspace(-3, 3, 7)})
    lr_all.fit(X_proc, Y)

    skf = StratifiedKFold(n_splits=5,shuffle=True,random_state = 42)
    true_concat = nans([n_trials,2])
    pred_concat = nans([n_trials,2])
    trial_save_idx = 0
    for training_set, test_set in skf.split(range(0,n_trials),cond_dict):
        #split training and testing by trials
        X_train, X_test, y_train, y_test = procX_train_test(X,Y,training_set,test_set)
        lr = GridSearchCV(Ridge(), {'alpha': np.logspace(-3, 3, 7)}) 
        lr.fit(X_train, y_train)
        y_test_predicted = lr.predict(X_test)

        n = y_test_predicted.shape[0]
        true_concat[trial_save_idx:trial_save_idx+n,:] = y_test
        pred_concat[trial_save_idx:trial_save_idx+n,:] = y_test_predicted
        trial_save_idx += n

    sses =get_sses_pred(true_concat,pred_concat)
    sses_mean=get_sses_mean(true_concat)
    R2 =1-np.sum(sses)/np.sum(sses_mean)     
    nan_cond_r2_arr[i] = R2

In [ ]:
nan_bump_r2_arr = nans([n_bins])
for i in range(n_bins):
    X = nan_trial_PCA[:,i,:]
    Y = nan_trial_ang[:,i,:]
    X_proc = (X - np.nanmean(X,axis=0))/np.nanstd(X,axis=0)
    lr_all = GridSearchCV(Ridge(), {'alpha': np.logspace(-3, 3, 7)})
    lr_all.fit(X_proc, Y)

    skf = StratifiedKFold(n_splits=5,shuffle=True,random_state = 42)
    true_concat = nans([n_trials,2])
    pred_concat = nans([n_trials,2])
    trial_save_idx = 0
    for training_set, test_set in skf.split(range(0,n_trials),cond_dict):
        #split training and testing by trials
        X_train, X_test, y_train, y_test = procX_train_test(X,Y,training_set,test_set)
        lr = GridSearchCV(Ridge(), {'alpha': np.logspace(-3, 3, 7)}) 
        lr.fit(X_train, y_train)
        y_test_predicted = lr.predict(X_test)

        n = y_test_predicted.shape[0]
        true_concat[trial_save_idx:trial_save_idx+n,:] = y_test
        pred_concat[trial_save_idx:trial_save_idx+n,:] = y_test_predicted
        trial_save_idx += n

    sses =get_sses_pred(true_concat,pred_concat)
    sses_mean=get_sses_mean(true_concat)
    R2 =1-np.sum(sses)/np.sum(sses_mean)     
    nan_bump_r2_arr[i] = R2

In [ ]:
# fb_nan_cond_r2 = nan_cond_r2_arr
# fb_nan_bump_r2 = nan_bump_r2_arr
cd_nan_cond_r2 = nan_cond_r2_arr
cd_nan_bump_r2 = nan_bump_r2_arr
# cd_fb_nan_cond_r2 = nan_cond_r2_arr
# cd_fb_nan_bump_r2 = nan_bump_r2_arr

In [ ]:
lag_axis = np.arange(-300,1000,50)+50
fig, ax = plt.subplots(figsize=(7.5,3))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.plot(lag_axis,cd_nan_cond_r2, "-o",color = 'green', label = 'CD')
plt.plot(lag_axis,fb_nan_cond_r2, "-o",color = 'magenta', label = 'FB')
plt.plot(lag_axis,cd_fb_nan_cond_r2, "-o",color = 'brown', label = 'CD+FB')

plt.xlabel('Time after movement onset (ms)'); plt.ylabel('R2'); plt.title('Reach direction r2')
plt.legend(fontsize=8)
plt.xlim([-210, 1010])
plt.ylim([-0.1, 1.0])
plt.axvline(0, color = 'k',linestyle = '--')
plt.tight_layout()
plt.show()

In [ ]:
lag_axis = np.arange(-300,1000,50)+50
fig, ax = plt.subplots(figsize=(7.5,3))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.plot(lag_axis,cd_nan_bump_r2, "-o",color = 'green', label = 'CD')
plt.plot(lag_axis,fb_nan_bump_r2, "-o",color = 'magenta', label = 'FB')
plt.plot(lag_axis,cd_fb_nan_bump_r2, "-o",color = 'brown', label = 'CD_FB')

plt.xlabel('Time after movement onset (ms)'); plt.ylabel('R2'); plt.title('Bump direction r2')
plt.legend(fontsize=8)
plt.xlim([-210, 1010])
plt.ylim([-0.1, 1.0])
plt.axvline(0, color = 'k',linestyle = '--')
plt.tight_layout()
plt.show()

## Active and Passive trials

In [ ]:
pred_range = (-300, 1000)
x_field = 'CD_FB_proj'
mask = active_mask
n_trials = active_n_trials
cond_dict = active_cond_dict
dim = dataset_50ms.data[x_field].shape[1]
active_df = dataset_50ms.make_trial_data(align_field='move_onset_time', align_range=pred_range, ignored_trials=~mask)
dirs = [dataset_50ms.trial_info[dataset_50ms.trial_info.trial_id == i]['cond_dir'] for i in active_df.trial_id]
cos_x = [round(math.cos(math.radians(i)),3) for i in dirs]
sin_y = [round(math.sin(math.radians(i)),3) for i in dirs]
cos_sin = np.array([cos_x, sin_y]).T
act_trial_PCA = active_df[x_field].to_numpy().reshape((n_trials, -1, dim))
print(act_trial_PCA.shape)
act_trial_ang = cos_sin.reshape((n_trials, -1, 2))
# act_trial_ang = np.array(cos_x).reshape((n_trials,-1,1))
# act_trial_ang = np.array(sin_y).reshape((n_trials,-1,1))

print(act_trial_ang.shape)

In [ ]:
# # decoders angle
# dataset = dataset_50ms
# x_field = 'PCA_40'
# y_field ='hand_acc'

# lag_axis = np.arange(-200,1000,50)+50
# lag_axis = np.arange(-200,1000,50)+50

# act_t_label = lag_axis
# act_X_coef_array = act_coefs_arr[:,0,:]
# angDist_array = nans([len(act_t_label),len(act_t_label)])

# for i in range(len(act_t_label)):
#     for j in range(len(act_t_label)):
#         angDist_array[i,j] = math.degrees(angle_between(act_X_coef_array[i,:],act_X_coef_array[j,:]))
# fig, ax = plt.subplots(figsize=(10, 10))
# im = ax.imshow(angDist_array)
# ax.set_xlabel('Active time (ms)')
# ax.set_ylabel('Active time (ms)')

# ax.set_xticks(np.arange(len(act_t_label)))
# ax.set_yticks(np.arange(len(act_t_label)))
# ax.set_xticklabels(labels=act_t_label,size=8)
# ax.set_yticklabels(labels=act_t_label,size=8)

# ax.set_title("Angle between CD+FB weights")

# for i in range(len(act_t_label)):
#     for j in range(len(act_t_label)):
#         text = ax.text(j, i, str(int(angDist_array[i, j])),
#                         ha="center", va="center", color="w", fontsize=14)
# plt.tight_layout()
# # figDir = '/Users/sherryan/area2_population_analysis/figures_plus/'
# # plt.savefig(figDir + monkey + '_decoder_angle.pdf', dpi = 'figure')
# plt.show()

In [44]:
n_bins = act_trial_PCA.shape[1]
act_coefs_arr = nans([n_bins, 2, dim])
act_offset_arr = nans([n_bins, 2])
act_r2_arr = nans([n_bins])
act_r2_xy_arr = nans([n_bins,2])

for i in range(n_bins):
    X = act_trial_PCA[:,i,:]
    Y = act_trial_ang[:,i,:]
    X_proc = (X - np.nanmean(X,axis=0))/np.nanstd(X,axis=0)
    lr_all = GridSearchCV(Ridge(), {'alpha': np.logspace(-3, 3, 7)})
    lr_all.fit(X_proc, Y)
    act_coefs_arr[i,:,:] = lr_all.best_estimator_.coef_
    act_offset_arr[i,:] = lr_all.best_estimator_.intercept_
    skf = StratifiedKFold(n_splits=5,shuffle=True,random_state = 42)
    true_concat = nans([n_trials,2])
    pred_concat = nans([n_trials,2])
    trial_save_idx = 0
    for training_set, test_set in skf.split(range(0,n_trials),cond_dict):
        #split training and testing by trials
        X_train, X_test, y_train, y_test = procX_train_test(X,Y,training_set,test_set)
        lr = GridSearchCV(Ridge(), {'alpha': np.logspace(-3, 3, 7)})
        lr.fit(X_train, y_train)
        y_test_predicted = lr.predict(X_test)

        n = y_test_predicted.shape[0]
        true_concat[trial_save_idx:trial_save_idx+n,:] = y_test
        pred_concat[trial_save_idx:trial_save_idx+n,:] = y_test_predicted
        trial_save_idx += n
    for j in range(true_concat.shape[1]):
        sses =get_sses_pred(true_concat[:,j],pred_concat[:,j])
        sses_mean=get_sses_mean(true_concat[:,j])
        act_r2_xy_arr[i,j] =1-np.sum(sses)/np.sum(sses_mean)   
    sses =get_sses_pred(true_concat,pred_concat)
    sses_mean=get_sses_mean(true_concat)
    R2 =1-np.sum(sses)/np.sum(sses_mean)     
    act_r2_arr[i] = R2

In [45]:
np.random.shuffle(act_trial_ang)
act_shuffle_r2_arr = nans([n_bins])
for i in range(n_bins):
    X = act_trial_PCA[:,i,:]
    Y = act_trial_ang[:,i,:]
    X_proc = (X - np.nanmean(X,axis=0))/np.nanstd(X,axis=0)
    lr_all = GridSearchCV(Ridge(), {'alpha': np.logspace(-3, 3, 7)})
    lr_all.fit(X_proc, Y)
    kf = KFold(n_splits=5,shuffle=True,random_state = 42)
    true_concat = nans([n_trials,2])
    pred_concat = nans([n_trials,2])
    trial_save_idx = 0
    for training_set, test_set in kf.split(range(0,n_trials)):
        #split training and testing by trials
        X_train, X_test, y_train, y_test = procX_train_test(X,Y,training_set,test_set)
        lr = GridSearchCV(Ridge(), {'alpha': np.logspace(-3, 3, 7)})
        lr.fit(X_train, y_train)
        y_test_predicted = lr.predict(X_test)
        n = y_test_predicted.shape[0]
        true_concat[trial_save_idx:trial_save_idx+n,:] = y_test
        pred_concat[trial_save_idx:trial_save_idx+n,:] = y_test_predicted
        trial_save_idx += n
    sses =get_sses_pred(true_concat,pred_concat)
    sses_mean=get_sses_mean(true_concat)
    R2 =1-np.sum(sses)/np.sum(sses_mean)     
    act_shuffle_r2_arr[i] = R2

In [ ]:
pred_range = (-300, 1000)
mask = passive_mask
n_trials = passive_n_trials
cond_dict = passive_cond_dict
passive_df = dataset_50ms.make_trial_data(align_field='move_onset_time', align_range=pred_range, ignored_trials=~mask)
dirs = [dataset_50ms.trial_info[dataset_50ms.trial_info.trial_id == i]['cond_dir'] for i in passive_df.trial_id]
cos_x = [round(math.cos(math.radians(i)),3) for i in dirs]
sin_y = [round(math.sin(math.radians(i)),3) for i in dirs]
cos_sin = np.array([cos_x, sin_y]).T
pas_trial_PCA = passive_df[x_field].to_numpy().reshape((n_trials, -1, dim))
print(pas_trial_PCA.shape)
pas_trial_ang = cos_sin.reshape((n_trials, -1, 2))
# pas_trial_ang = np.array(cos_x).reshape((n_trials,-1,1))
# pas_trial_ang = np.array(sin_y).reshape((n_trials,-1,1))

print(pas_trial_ang.shape)

In [47]:
pas_coefs_arr = nans([n_bins, 2, dim])
pas_offset_arr = nans([n_bins, 2])
pas_r2_arr = nans([n_bins])
pas_r2_xy_arr = nans([n_bins,2])

for i in range(n_bins):
    X = pas_trial_PCA[:,i,:]
    Y = pas_trial_ang[:,i,:]
    X_proc = (X - np.nanmean(X,axis=0))/np.nanstd(X,axis=0)
    lr_all = GridSearchCV(Ridge(), {'alpha': np.logspace(-3, 3, 7)})
    lr_all.fit(X_proc, Y)
    pas_coefs_arr[i,:,:] = lr_all.best_estimator_.coef_
    pas_offset_arr[i,:] = lr_all.best_estimator_.intercept_
    skf = StratifiedKFold(n_splits=5,shuffle=True,random_state = 42)
    true_concat = nans([n_trials,2])
    pred_concat = nans([n_trials,2])
    trial_save_idx = 0
    for training_set, test_set in skf.split(range(0,n_trials),cond_dict):
        #split training and testing by trials
        X_train, X_test, y_train, y_test = procX_train_test(X,Y,training_set,test_set)
        lr = GridSearchCV(Ridge(), {'alpha': np.logspace(-3, 3, 7)})
        lr.fit(X_train, y_train)
        y_test_predicted = lr.predict(X_test)

        n = y_test_predicted.shape[0]
        true_concat[trial_save_idx:trial_save_idx+n,:] = y_test
        pred_concat[trial_save_idx:trial_save_idx+n,:] = y_test_predicted
        trial_save_idx += n
    for j in range(true_concat.shape[1]):
        sses =get_sses_pred(true_concat[:,j],pred_concat[:,j])
        sses_mean=get_sses_mean(true_concat[:,j])
        pas_r2_xy_arr[i,j] =1-np.sum(sses)/np.sum(sses_mean)   
    sses =get_sses_pred(true_concat,pred_concat)
    sses_mean=get_sses_mean(true_concat)
    R2 =1-np.sum(sses)/np.sum(sses_mean)     
    pas_r2_arr[i] = R2

In [ ]:
# np.random.shuffle(pas_trial_ang)
# pas_shuffle_r2_arr = nans([n_bins])
# for i in range(n_bins):
#     X = pas_trial_PCA[:,i,:]
#     Y = pas_trial_ang[:,i,:]
#     X_proc = (X - np.nanmean(X,axis=0))/np.nanstd(X,axis=0)
#     lr_all = GridSearchCV(Ridge(), {'alpha': np.logspace(-3, 3, 7)})
#     lr_all.fit(X_proc, Y)
#     kf = KFold(n_splits=5,shuffle=True,random_state = 42)
#     true_concat = nans([n_trials,2])
#     pred_concat = nans([n_trials,2])
#     trial_save_idx = 0
#     for training_set, test_set in kf.split(range(0,n_trials)):
#         #split training and testing by trials
#         X_train, X_test, y_train, y_test = procX_train_test(X,Y,training_set,test_set)
#         lr = GridSearchCV(Ridge(), {'alpha': np.logspace(-3, 3, 7)}) 
#         lr.fit(X_train, y_train)
#         y_test_predicted = lr.predict(X_test)

#         n = y_test_predicted.shape[0]
#         true_concat[trial_save_idx:trial_save_idx+n,:] = y_test
#         pred_concat[trial_save_idx:trial_save_idx+n,:] = y_test_predicted
#         trial_save_idx += n

#     sses =get_sses_pred(true_concat,pred_concat)
#     sses_mean=get_sses_mean(true_concat)
#     R2 =1-np.sum(sses)/np.sum(sses_mean)     
#     pas_shuffle_r2_arr[i] = R2

In [48]:
# cd_act_r2 = act_r2_arr
# cd_pas_r2 = pas_r2_arr
# fb_act_r2 = act_r2_arr
# fb_pas_r2 = pas_r2_arr
cd_fb_act_r2 = act_r2_arr
cd_fb_pas_r2 = pas_r2_arr

# neural_pc_act_r2 = act_r2_arr
# neural_pc_pas_r2 = pas_r2_arr
# neural_spikes_act_r2 = act_r2_arr
# neural_spikes_pas_r2 = pas_r2_arr
# emg_act_r2 = act_r2_arr
# emg_pas_r2 = pas_r2_arr
# muscle_pc_act_r2 = act_r2_arr
# muscle_pc_pas_r2 = pas_r2_arr
# joint_pc_act_r2 = act_r2_arr
# joint_pc_pas_r2 = pas_r2_arr


np.savez(monkey+'_cdfb_dir_r2_100oneside', 
         cd_act_r2 = cd_act_r2, cd_pas_r2 = cd_pas_r2, \
         fb_act_r2 = fb_act_r2,fb_pas_r2 = fb_pas_r2, \
         cd_fb_act_r2 = cd_fb_act_r2,cd_fb_pas_r2 = cd_fb_pas_r2)

        #  neural_pc_act_r2 = neural_pc_act_r2, neural_pc_pas_r2 = neural_pc_pas_r2)
        #  neural_spikes_act_r2 = neural_spikes_act_r2, neural_spikes_pas_r2 = neural_spikes_pas_r2,\
        #  emg_act_r2 = emg_act_r2, emg_pas_r2 = emg_pas_r2,\
        #  muscle_pc_act_r2 = act_r2_arr, muscle_pc_pas_r2 = pas_r2_arr,\
        #  joint_pc_act_r2 = act_r2_arr, joint_pc_pas_r2 = pas_r2_arr)

In [ ]:
lag_axis = np.arange(-300,1000,50)
fig, ax = plt.subplots(figsize=(10,4))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.plot(lag_axis,cd_act_r2, "-o",color = 'green', label = 'CD')
plt.plot(lag_axis,fb_act_r2, "-o",color = 'magenta', label = 'FB')
plt.plot(lag_axis,cd_fb_act_r2, "-o",color = 'brown', label = 'CD_FB')
# plt.plot(lag_axis,neural_pc_act_r2, "-o",color = 'grey', label = 'Neural PC')
# plt.plot(np.arange(-150,1000,50)+50,neural_spikes_act_r2, "-o",color = 'k', label = 'Neural')
# plt.plot(lag_axis,emg_act_r2, "-o",color = 'orange', label = 'EMG')
# plt.plot(lag_axis,muscle_pc_act_r2, "-o", label = 'muscle PC')
# plt.plot(lag_axis,joint_pc_act_r2, "-o", color = 'red',label = 'joint PC')



plt.plot(lag_axis,act_shuffle_r2_arr, color = 'k', ls='--',label = 'shuffle')
plt.xlabel('Time after movement onset (ms)'); plt.ylabel('R2'); plt.title('Active direction r2')
plt.legend(fontsize=8)
plt.xlim([-210, 1010])
plt.ylim([-0.1, 1.0])
plt.axvline(0, color = 'k',linestyle = '--')
plt.tight_layout()
# plt.savefig(figDir + monkey + '_cdfb_dir_r2_active_.pdf',dpi = 'figure')
plt.show()

In [ ]:
lag_axis = np.arange(-300,1000,50)
fig, ax = plt.subplots(figsize=(10,4))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.plot(lag_axis,cd_pas_r2, "-o",color = 'green', label = 'CD')
plt.plot(lag_axis,fb_pas_r2, "-o",color = 'magenta', label = 'FB')
plt.plot(lag_axis,cd_fb_pas_r2, "-o",color = 'brown', label = 'CD_FB')
# plt.plot(lag_axis,neural_pc_pas_r2, "-o",color = 'grey', label = 'Neural PC')
# plt.plot(np.arange(-150,1000,50)+50,neural_spikes_pas_r2, "-o",color = 'k', label = 'Neural')
# plt.plot(lag_axis,emg_pas_r2, "-o",color = 'orange', label = 'EMG')
# plt.plot(lag_axis,muscle_pc_pas_r2, "-o", label = 'muscle PC')
# plt.plot(lag_axis,joint_pc_pas_r2, "-o", color = 'red',label = 'joint PC')


plt.plot(lag_axis,act_shuffle_r2_arr, color = 'k', ls='--',label = 'shuffle')
plt.xlabel('Time after movement onset (ms)'); plt.ylabel('R2'); plt.title(' Passive direction r2')
plt.legend(fontsize=8)
plt.xlim([-210, 1010])
plt.ylim([-0.1, 1.0])
plt.axvline(0, color = 'k',linestyle = '--')
plt.tight_layout()
# plt.savefig(figDir + monkey + '_cdfb_dir_r2_passive.pdf',dpi = 'figure')
plt.show()

In [100]:
# neural_pc_act_r2 = act_r2_arr
# neural_pc_pas_r2 = pas_r2_arr
# neural_pc_act_xy_r2 = act_r2_xy_arr
# neural_pc_pas_xy_r2 = pas_r2_xy_arr
# vel_act_r2 = act_r2_arr
# vel_pas_r2 = pas_r2_arr
vel_act_xy_r2 = act_r2_xy_arr

In [ ]:
lag_axis = np.arange(-300,1000,50)
fig, ax = plt.subplots(figsize=(10,6))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.plot(lag_axis,neural_pc_act_r2, "-o",color = 'k', label = 'Active, 20PC')
plt.plot(lag_axis,neural_pc_act_xy_r2[:,0], "-o",label = 'Active, 20PC,x-dir')
plt.plot(lag_axis,neural_pc_act_xy_r2[:,1], "-o", label = 'Active, 20PC,y-dir')
plt.plot(lag_axis,vel_act_r2,"-o", color = 'gray',label = 'Active, velocity')
plt.plot(lag_axis,vel_act_xy_r2[:,0], "-o",color = 'gray',label = 'Active, velocity,x-dir')
plt.plot(lag_axis,vel_act_xy_r2[:,1], "-o", color = 'gray',label = 'Active, velocity,y-dir')

# plt.plot(lag_axis,neural_pc_pas_r2,"-o", color = 'red', label = 'Passive, 20PC')
# plt.plot(lag_axis,pas_shuffle_r2_arr, color = 'red', ls='--',label = 'Passive shuffle')
# plt.plot(lag_axis,vel_pas_r2,"-o", color = 'gray',label = 'Passive, velocity')

plt.plot(lag_axis,act_shuffle_r2_arr, color = 'k', ls='--',label = 'shuffle')

plt.xlabel('Time after movement onset (ms)'); plt.ylabel('R2'); 
plt.title('Direction r2')
plt.legend(fontsize=8)
plt.xlim([-300, 610])
plt.axvline(0, color = 'k',linestyle = '--')
plt.tight_layout()
# plt.savefig(figDir + monkey + '_direction_r2.pdf',dpi = 'figure')
plt.show()

# Single-lag decoding

In [ ]:
dataset = dataset_10ms
dataset.data.keys().unique(0)

In [ ]:
x_field = 'spikes_smth_40_oneside'
data = np.load(monkey+'_ac2_cdfb_data_'+x_field+'.npz')
data.files
dataset.add_continuous_data(data['CD_FB_proj'],'CD_FB_proj')
dataset.add_continuous_data(data['CD_proj'],'CD_proj')
dataset.add_continuous_data(data['FB_proj'],'FB_proj')
dataset.data.keys().unique(0)

In [ ]:
data = np.load(monkey+'_cdfb_weights_spikes_smth_40.npz')
data.files

In [ ]:
# CD_FB_axes = np.vstack([CD_axes,FB_axes])
# CD_FB_axes.shape

CD_FB_axes = np.vstack([data['CD_axes'],data['FB_axes']])
CD_FB_axes.shape

In [ ]:
_, best_coef,_, _, _ = fit_and_predict(dataset, active_mask, 'move_onset_time',pred_range, 0, 'spikes_smth_40', 'hand_vel', cond_dict = cond_dict)
X = CD_FB_axes.T #(n_samples, n_features)
Y = best_coef.T
reg = LinearRegression().fit(X, Y)
reg.score(X,Y)
reg.coef_.shape

In [ ]:
reg.score(X,Y)

In [ ]:
plt.plot(np.arange(1,reg.coef_.shape[1]+1),reg.coef_[1,:])
plt.title('y weights')

In [ ]:
plt.plot(np.arange(1,reg.coef_.shape[1]+1),reg.coef_[0,:])
plt.title('x weights')

## Straight decoding

In [ ]:
dataset_10ms.data.keys().unique(0)

In [ ]:
x_field = 'ac150_CD_FB_proj_spikes_smth_150_oneside'
y_field ='hand_vel'
lag_axis = np.arange(-300,300,20)
pred_range = (-100, 1000)
trial_mask = active_mask
cond_dict = active_cond_dict
# pred_range = (-100, 120)
# trial_mask = passive_mask
# cond_dict = passive_cond_dict
dataset=dataset_10ms
dim = dataset.data[x_field].shape[1]
print(dim)

In [ ]:
r2_array = nans([len(lag_axis)]); r2_feature_array = nans([len(lag_axis),dataset.data[y_field].shape[1]])
r_array = nans([len(lag_axis)])
coef_array = nans([len(lag_axis),dataset.data[y_field].shape[1],dim])
for i in range(len(lag_axis)):
    lag = lag_axis[i]
    r2, coef,_,vel_df,r2_arr = fit_and_predict(dataset, trial_mask, 'move_onset_time',pred_range, lag, x_field, y_field,cond_dict)
    # r2, vel_df,r2_arr = fit_and_predict_DNN(dataset, trial_mask, 'move_onset_time',pred_range, lag, x_field, y_field,cond_dict)
    r2_array[i] = r2; r2_feature_array[i,:] = r2_arr
    r = scipy.stats.pearsonr(vel_df[y_field].to_numpy().reshape(-1), vel_df['pred_vel'].to_numpy().reshape(-1))[0]
    r_array[i] = r
    coef_array[i,:,:] = coef
time_max = lag_axis[np.argmax(r2_array)]
print(np.max(r2_array))
print(time_max)
# _, best_coef,best_intercept, best_vel_df, r2_arr = fit_and_predict(dataset, trial_mask, 'move_onset_time',pred_range, time_max, x_field, y_field, cond_dict = cond_dict)

In [ ]:
plt.plot(lag_axis, r2_array)
plt.plot(lag_axis, r2_feature_array[:,0],label='x')
plt.plot(lag_axis, r2_feature_array[:,1],label='y')
print(lag_axis[np.argmax(r2_feature_array[:,0])])

In [ ]:
x_time_max = lag_axis[np.argmax(r2_feature_array[:,0])]
y_time_max = lag_axis[np.argmax(r2_feature_array[:,1])]
print('x',x_time_max)
print('y',y_time_max)
if x_time_max != y_time_max:
    _, x_best_coef,x_best_intercept, _, _ = fit_and_predict(dataset, trial_mask, 'move_onset_time',pred_range, x_time_max, x_field, y_field, cond_dict = cond_dict)
    _, y_best_coef,y_best_intercept, _, _ = fit_and_predict(dataset, trial_mask, 'move_onset_time',pred_range, y_time_max, x_field, y_field, cond_dict = cond_dict)
# r2_feature_array.shape

In [ ]:
# n_cd_dims = 4
# if x_time_max == y_time_max:
#     x_cd_weight = np.sum(abs(best_coef[0,:n_cd_dims]))
#     x_fb_weight = np.sum(abs(best_coef[0,n_cd_dims:]))
#     y_cd_weight = np.sum(abs(best_coef[1,:n_cd_dims]))
#     y_fb_weight = np.sum(abs(best_coef[1,n_cd_dims:]))
# else:
#     x_cd_weight = np.sum(abs(x_best_coef[0,:n_cd_dims]))
#     x_fb_weight = np.sum(abs(x_best_coef[0,n_cd_dims:]))
#     y_cd_weight = np.sum(abs(y_best_coef[1,:n_cd_dims]))
#     y_fb_weight = np.sum(abs(y_best_coef[1,n_cd_dims:]))
# print(x_cd_weight)
# print(x_fb_weight)
# print(y_cd_weight)
# print(y_fb_weight)

In [ ]:
# r2_cross_array = nans([len(lag_axis)]); x_r2_cross_array = nans([len(lag_axis)])
# r_cross_array = nans([len(lag_axis)])
# for i in range(len(lag_axis)):
#     lag = lag_axis[i]
#     r2, _, x_r2,_,vel_df = pred_with_new_weights(dataset, trial_mask, 'move_onset_time',pred_range, lag,x_field,
#                                                 y_field, x_best_coef, x_best_intercept, 'move_onset_time',pred_range, (pred_range[0]+x_time_max, pred_range[1]+x_time_max), trial_mask)
#     r2_cross_array[i] = r2; x_r2_cross_array[i] = x_r2
#     r = scipy.stats.pearsonr(vel_df[y_field].to_numpy().reshape(-1), vel_df['pred_vel'].to_numpy().reshape(-1))[0]
#     r_cross_array[i] = r
# plt.plot(lag_axis,r_cross_array)
# plt.xlabel('Time lag (ms)')
# plt.ylabel('r')

In [ ]:
# idx_max_pos = np.argwhere(r2_array == np.max(r2_array[np.argwhere(lag_axis==0)[0,0]:]))[0,0]
# idx_max_neg = np.argwhere(r2_array == np.max(r2_array[:np.argwhere(lag_axis==0)[0,0]]))[0,0]
# print(lag_axis[idx_max_pos])
# print(lag_axis[idx_max_neg])

# #For velocity, override max identification
# # idx_max_pos = np.argwhere(lag_axis==80)[0,0]
# # idx_max_neg = np.argwhere(lag_axis==-40)[0,0]

# ang_to_max_x = nans([len(lag_axis)])
# ang_to_max_y = nans([len(lag_axis)])
# # ang_to_max_z = nans([len(lag_axis)])
# for i in range(0, len(coef_array)):
#     ang_to_max_x[i] = math.degrees(angle_between(coef_array[i,0,:],coef_array[idx_max_neg,0,:]))
#     ang_to_max_y[i] = math.degrees(angle_between(coef_array[i,1,:],coef_array[idx_max_neg,1,:]))
#     # ang_to_max_z[i] = math.degrees(angle_between(coef_array[i,2,:],coef_array[idx_max_neg,2,:]))
# fig, ax = plt.subplots()
# ax.spines['right'].set_visible(False)
# ax.spines['top'].set_visible(False)
# # plt.ylim([-5, 130])
# plt.xlim([-310, 310])
# plt.scatter(lag_axis, ang_to_max_x,label = 'x',color = 'green')
# plt.scatter(lag_axis, ang_to_max_y,label = 'y',color = 'blue')
# # plt.scatter(lag_axis, ang_to_max_z,label = 'wrist_abduction',color = 'orange')

# plt.legend()
# plt.xlabel('Time lag (ms)')
# plt.ylabel('Angle (degrees)')
# mean = np.mean([ang_to_max_x[idx_max_neg], ang_to_max_y[idx_max_neg]])
# print(mean)
# # plt.vlines(lag_axis[idx_max_pos],-5, mean, color = 'k',linestyle="dashed")
# # plt.hlines(mean, -310, lag_axis[idx_max_pos], color = 'k',linestyle="dashed")
# plt.tight_layout()
# # plt.savefig(figDir + monkey + label + str(0) +'_angle.pdf', dpi = 'figure')
# plt.show()


In [ ]:
pred_range_arr = [(-100, 0),(0, 200),(200, 400),(400, 600),(600, 800),(800, 1000)]
# pred_range_arr = [(-100, 0),(0, 100),(100, 200),(200, 300),(300, 400),(400, 500)]

r2_array_tw = nans([len(pred_range_arr)]); r2_feature_array_tw = nans([len(pred_range_arr),dataset.data[y_field].shape[1]])
for tw in range(len(pred_range_arr)):
    tw_range = pred_range_arr[tw]
    if x_time_max == y_time_max:
        r2, _, x_r2,y_r2,_ = pred_with_new_weights(dataset, trial_mask, 'move_onset_time',tw_range, time_max,x_field,
                                                y_field, best_coef, best_intercept, 'move_onset_time',pred_range, (pred_range[0]+time_max, pred_range[1]+time_max), trial_mask)
        r2_array_tw[tw] = r2; r2_feature_array_tw[tw,:] = np.array([x_r2,y_r2])
    else:
        r2, _, _,_,_ = pred_with_new_weights(dataset, trial_mask, 'move_onset_time',tw_range, time_max,x_field,
                                                y_field, best_coef, best_intercept, 'move_onset_time',pred_range, (pred_range[0]+time_max, pred_range[1]+time_max), trial_mask)
        _, _, x_r2,_,_ = pred_with_new_weights(dataset, trial_mask, 'move_onset_time',tw_range, x_time_max,x_field,
                                                y_field, x_best_coef, x_best_intercept, 'move_onset_time',pred_range, (pred_range[0]+x_time_max, pred_range[1]+x_time_max), trial_mask)
        _, _, _,y_r2,_ = pred_with_new_weights(dataset, trial_mask, 'move_onset_time',tw_range, y_time_max,x_field,
                                        y_field, y_best_coef, y_best_intercept, 'move_onset_time',pred_range, (pred_range[0]+y_time_max, pred_range[1]+y_time_max), trial_mask)
        r2_array_tw[tw] = r2; r2_feature_array_tw[tw,:] = np.array([x_r2,y_r2])

In [ ]:
# Prepare for plotting
plot_dir = [0.0, 90.0, 180.0, 270.0] # limit plot directions to reduce cluttering
colors = ['red', 'blue', 'green', 'orange']
plot_dim = 'x' # plot x velocity 

x_axis = np.arange(-100,1000,dataset.bin_width)
fig, ax = plt.subplots()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
for trial_dir, color in zip(plot_dir, colors):
    cond_ids = dataset.trial_info[dataset.trial_info.cond_dir %360 == trial_dir].trial_id
    for _, trial in vel_df[np.isin(vel_df.trial_id, cond_ids)].groupby('trial_id'):
        plt.plot(x_axis, trial[y_field][plot_dim], color=color, linewidth=0.5)
        # plt.plot(x_axis, trial[y_field].to_numpy()[:,0], color=color, linewidth=0.5)
plt.xlabel('Time after movement onset (ms)')
# plt.ylabel('Hand velocity (cm/s)')

# plt.xlim([-100,500])
# plt.ylim([-0.65,0.65])
plt.tight_layout()
# plt.savefig(figDir + monkey + label + 'true.pdf', dpi = 'figure')
plt.show()

fig, ax = plt.subplots()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
for trial_dir, color in zip(plot_dir, colors):
    cond_ids = dataset.trial_info[dataset.trial_info.cond_dir %360 == trial_dir].trial_id
    for _, trial in sub_vel_df[np.isin(sub_vel_df.trial_id, cond_ids)].groupby('trial_id'):
         plt.plot(x_axis, trial['pred_vel'][plot_dim], color=color, linewidth=0.5)
        # plt.plot(x_axis, trial.pred_vel.to_numpy()[:,0], color=color, linewidth=0.5)
plt.xlabel('Time after movement onset (ms)')
# plt.ylabel('Hand velocity (cm/s)')
# plt.xlim([-100,500])
# plt.ylim([-0.65,0.65])
plt.tight_layout()
# plt.savefig(figDir + monkey + label + str(0) +'_pred.pdf', dpi = 'figure')
plt.show()

In [ ]:
# Prepare for plotting
plot_dir = [0.0, 90.0, 180.0, 270.0] # limit plot directions to reduce cluttering
colors = ['red', 'blue', 'green', 'orange']
plot_dim = 'y' # plot x velocity 

x_axis = np.arange(-100,1000,dataset.bin_width)
fig, ax = plt.subplots()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
for trial_dir, color in zip(plot_dir, colors):
    cond_ids = dataset.trial_info[dataset.trial_info.cond_dir %360 == trial_dir].trial_id
    for _, trial in vel_df[np.isin(vel_df.trial_id, cond_ids)].groupby('trial_id'):
        plt.plot(x_axis, trial[y_field][plot_dim], color=color, linewidth=0.5)
        # plt.plot(x_axis, trial[y_field].to_numpy()[:,0], color=color, linewidth=0.5)
plt.xlabel('Time after movement onset (ms)')
# plt.ylabel('Hand velocity (cm/s)')

# plt.xlim([-100,500])
# plt.ylim([-0.65,0.65])
plt.tight_layout()
# plt.savefig(figDir + monkey + label + 'true.pdf', dpi = 'figure')
plt.show()

fig, ax = plt.subplots()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
for trial_dir, color in zip(plot_dir, colors):
    cond_ids = dataset.trial_info[dataset.trial_info.cond_dir %360 == trial_dir].trial_id
    for _, trial in sub_vel_df[np.isin(sub_vel_df.trial_id, cond_ids)].groupby('trial_id'):
         plt.plot(x_axis, trial['pred_vel'][plot_dim], color=color, linewidth=0.5)
        # plt.plot(x_axis, trial.pred_vel.to_numpy()[:,0], color=color, linewidth=0.5)
plt.xlabel('Time after movement onset (ms)')
# plt.ylabel('Hand velocity (cm/s)')
# plt.xlim([-100,500])
# plt.ylim([-0.65,0.65])
plt.tight_layout()
# plt.savefig(figDir + monkey + label + str(0) +'_pred.pdf', dpi = 'figure')
plt.show()

In [ ]:
#Plot some periods of predictions
plot_dim='y'
fig, ax = plt.subplots(figsize=(8,3))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.plot(vel_df['hand_vel']['y'][1000:5000],color = 'k')
plt.plot(vel_df['pred_vel']['y'][1000:5000],color = 'brown')
plt.tight_layout()
plt.xlabel('Time after movement onset (ms)')
plt.ylabel('Hand velocity (cm/s)')
# plt.savefig(figDir + monkey+'_cd_pred_.pdf',dpi = 'figure')
plt.show()

In [ ]:
# dim= np.min([10, dataset.data[x_field].shape[1]])
plt.plot(lag_axis,r2_feature_array)
plt.legend(fontsize=8)
plt.xlabel('Time lag (ms)')
plt.ylabel('Variable R2')
plt.title(x_field+' -> '+y_field)
plt.show()

fig, ax = plt.subplots()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.plot(lag_axis, r2_array)
plt.title(x_field)

# for i in range(dim):
#     plt.plot(lag_axis,np.sum(abs(coef_array[:,:,order[i]]),axis=1),label = str(i+1))
# plt.legend(fontsize=8)
# plt.xlabel('Time lag (ms)')
# plt.ylabel('Latent weight')
# plt.title(x_field+' sum')
# plt.show()

# for i in range(dim):
#     plt.plot(lag_axis,(coef_array[:,0,order[i]]),label = str(i+1))
# plt.legend(fontsize=8)
# plt.xlabel('Time lag (ms)')
# plt.ylabel('Latent weight')
# plt.title(x_field+' x-dir')
# plt.show()
# for i in range(dim):
#     plt.plot(lag_axis,(coef_array[:,1,order[i]]),label = str(i+1))
# plt.legend(fontsize=8)
# plt.xlabel('Time lag (ms)')
# plt.ylabel('Latent weight')
# plt.title(x_field+' y-dir')

In [50]:
# x_r2_cd_only = r2_feature_array[:,0]
# y_r2_cd_only = r2_feature_array[:,1]
# r2_cd_only = r2_array
# tmax_cd_only = time_max

# x_r2_fb_only = r2_feature_array[:,0]
# y_r2_fb_only = r2_feature_array[:,1]
# r2_fb_only = r2_array
# tmax_fb_only = time_max

x_r2_cd_fb = r2_feature_array[:,0]
y_r2_cd_fb = r2_feature_array[:,1]
r2_cd_fb = r2_array
tmax_cd_fb = time_max

# x_r2_all_pc = r2_feature_array[:,0]
# y_r2_all_pc = r2_feature_array[:,1]
# r2_all_pc = r2_array
# tmax_all_pc = time_max

In [ ]:
np.savez(monkey+'_ac150_spikes_'+y_field +'_act_r2s', \
         x_r2_cd_only = x_r2_cd_only, y_r2_cd_only = y_r2_cd_only, r2_cd_only = r2_cd_only, \
         x_r2_fb_only = x_r2_fb_only, y_r2_fb_only = y_r2_fb_only, r2_fb_only = r2_fb_only,\
         x_r2_cd_fb = x_r2_cd_fb, y_r2_cd_fb = y_r2_cd_fb, r2_cd_fb = r2_cd_fb)
# ,
        #  x_r2_all_pc = x_r2_all_pc, y_r2_all_pc = y_r2_all_pc,r2_all_pc=r2_all_pc) 
# monkey = "Han_20171207"
data = np.load(monkey+'_ac150_spikes_hand_vel_act_r2s.npz')
data.files

In [ ]:
np.savez(monkey+'_all_nrn_150oneside'+y_field +'_act_r2s', \
        r2_all_nrn_array = r2_array,\
        r2_feature_all_nrn_array = r2_feature_array)

data_all_nrn = np.load(monkey+'_all_nrn_150onesidehand_vel_act_r2s.npz')
data_all_nrn.files

In [ ]:
lag_axis = np.arange(-300,300,20)
fig, ax = plt.subplots()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.axvline(0, color = 'k', linestyle='--')

plt.plot(lag_axis, data['r2_cd_only'],color = 'green',label='cd')
plt.plot(lag_axis, data['r2_fb_only'],color = 'magenta',label='fb')
plt.plot(lag_axis, data['r2_cd_fb'],color = 'brown',label='cd+fb')
# plt.plot(lag_axis, data['r2_all_pc'],color = 'k',label='20 PC')
plt.plot(lag_axis, data_all_nrn['r2_all_nrn_array'],color = 'k',label='all nrn')
plt.xlabel('Time lag (ms)')
plt.ylabel('R2')

In [ ]:
lag_axis[np.argmax(data['r2_cd_only'])]

In [ ]:
lag_axis = np.arange(-300,300,20)
fig, ax = plt.subplots()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.plot(lag_axis, data['x_r2_cd_only'],color = 'green',label='cd')
plt.plot(lag_axis, data['x_r2_fb_only'],color = 'magenta',label='fb')
plt.plot(lag_axis, data['x_r2_cd_fb'],color = 'brown',label='cd+fb')
# plt.plot(lag_axis,  data['x_r2_all_pc'],color = 'k',label='20 PC')
plt.plot(lag_axis, data_all_nrn['r2_feature_all_nrn_array'][:,0],color = 'k',label='all nrn')

plt.axvline(0, color = 'k', linestyle='--')
# print(time_max)
plt.xlabel('Time lag (ms)')
plt.ylabel('R2')
# plt.title(monkey + x_field+' x-dir')
# plt.ylim([0.05,0.8])
# plt.legend()
plt.tight_layout()
# plt.savefig(figDir + monkey +'_act_vel_y_.pdf', dpi = 'figure')
plt.show()

fig, ax = plt.subplots()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.plot(lag_axis, data['y_r2_cd_only'],color = 'green',label='cd')
plt.plot(lag_axis, data['y_r2_fb_only'],color = 'magenta',label='fb')
plt.plot(lag_axis, data['y_r2_cd_fb'],color = 'brown',label='cd+fb')
# plt.plot(lag_axis,  data['y_r2_all_pc'],color = 'k',label='20 PC')
plt.plot(lag_axis, data_all_nrn['r2_feature_all_nrn_array'][:,1],color = 'k',label='all nrn')

plt.axvline(0, color = 'k', linestyle='--')
# print(time_max)
plt.xlabel('Time lag (ms)')
plt.ylabel('R2')
# plt.title(monkey + x_field+' y-dir')
# plt.ylim([0.05,0.8])
# plt.legend()
plt.tight_layout()
# plt.savefig(figDir + monkey +'_act_vel_y_.pdf', dpi = 'figure')
plt.show()

## Dynamic weighting

In [ ]:
x_field = '8020_CD_FB_proj_spikes_smth_40'
y_field ='hand_vel'
lag_axis = np.arange(-300,300,20)
pred_range = (-100, 1000)
trial_mask = active_mask
cond_dict = active_cond_dict
# pred_range = (-100, 500)
# trial_mask = passive_mask
# cond_dict = passive_cond_dict

dim = dataset.data[x_field].shape[1]
print(dim)

In [ ]:
pred_range_arr = [(-100, 0),(0, 200),(200, 400),(400, 600),(600, 800),(800, 1000)]
# pred_range_arr = [(-100, 0),(0, 100),(100, 200),(200, 300),(300, 400),(400, 500)]

r2_array_dynam = nans([len(pred_range_arr),len(lag_axis)]); r2_feature_array_dynam = nans([len(pred_range_arr),len(lag_axis),dataset.data[y_field].shape[1]])
coef_array_dynam = nans([len(pred_range_arr),len(lag_axis),dataset.data[y_field].shape[1],dim])
time_max_array_dynam = nans([len(pred_range_arr)])
vel_df_array_dynam = []
for tw in range(len(pred_range_arr)):
    tw_range = pred_range_arr[tw]
    for i in range(len(lag_axis)):
        lag = lag_axis[i]
        r2, coef,_,vel_df,r2_arr = fit_and_predict(dataset, trial_mask, 'move_onset_time',tw_range, lag, x_field, y_field,cond_dict)
        r2_array_dynam[tw,i] = r2; r2_feature_array_dynam[tw,i,:] = r2_arr
        coef_array_dynam[tw,i,:,:] = coef
    t_max = lag_axis[np.argmax(r2_array_dynam[tw,:])]
    print(np.max(r2_array_dynam[tw,:]))
    time_max_array_dynam[tw] = t_max
    _, _,_, vel_df, _ = fit_and_predict(dataset, trial_mask, 'move_onset_time',tw_range, t_max, x_field, y_field, cond_dict = cond_dict)
    vel_df_array_dynam.append(np.array(vel_df['pred_vel']))

In [ ]:
fig, ax = plt.subplots()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.plot(range(len(pred_range_arr)), r2_array_tw,'k',label='static')
plt.plot(range(len(pred_range_arr)), r2_feature_array_tw[:,0],'blue')
plt.plot(range(len(pred_range_arr)), r2_feature_array_tw[:,1],'orange')
plt.plot(range(len(pred_range_arr)), np.max(r2_array_dynam,axis=1),'k--',label='dynamic')
plt.plot(range(len(pred_range_arr)), np.max(r2_feature_array_dynam[:,:,0],axis=1),'blue',ls='--')
plt.plot(range(len(pred_range_arr)), np.max(r2_feature_array_dynam[:,:,1],axis=1),'orange',ls='--')
plt.xticks(range(len(pred_range_arr)),['-100','0','200','400','600','800'])
plt.ylim([-0.1,1])
plt.xlabel('Time window start (ms)')
plt.ylabel('R2')
plt.legend()
plt.show()
plt.plot(time_max_array_dynam,range(len(pred_range_arr)),'k--')
plt.axvline(x=time_max,color='k')
plt.yticks(range(len(pred_range_arr)),['-100','0','200','400','600','800'])
plt.xlabel('Best time lag (ms)')
plt.ylabel('Time window start (ms)')


In [ ]:
stack_array = []
for i in range(len(vel_df_array_dynam)):
    stack_array.append(vel_df_array_dynam[i].reshape(active_n_trials,-1,2))
concat_pred = np.hstack(stack_array)
print(concat_pred.shape)

In [ ]:
all_dir = np.array([0.0, 45.0,90.0, 135.0, 180.0, 225.0, 270.0, 315.0])
plot_dir = [0.0, 90.0, 180.0, 270.0] # limit plot directions to reduce cluttering
colors = ['red', 'blue', 'green', 'orange']
for i in range(len(plot_dir)):
    idx = np.argwhere(all_dir==plot_dir[i])[0,0]
    plt.plot(np.arange(-100, 1000, 10),concat_pred[active_cond_dict==idx,:,0].T,color=colors[i],alpha=0.5,linewidth=1)
plt.xlabel('Time after move onset (ms)')
plt.ylabel('Hand velocity')

In [ ]:
all_dir = np.array([0.0, 45.0,90.0, 135.0, 180.0, 225.0, 270.0, 315.0])
plot_dir = [0.0, 90.0, 180.0, 270.0] # limit plot directions to reduce cluttering
colors = ['red', 'blue', 'green', 'orange']
for i in range(len(plot_dir)):
    idx = np.argwhere(all_dir==plot_dir[i])[0,0]
    plt.plot(np.arange(-100, 1000, 10),concat_pred[active_cond_dict==idx,:,1].T,color=colors[i],alpha=0.5,linewidth=1)
plt.xlabel('Time after move onset (ms)')
plt.ylabel('Hand velocity')

In [ ]:
fig, ax = plt.subplots()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
for i in range(len(pred_range_arr)):
    plt.plot(lag_axis, r2_array_dynam[i,:].T,label=pred_range_arr[i])
plt.legend(fontsize=8)
plt.xlabel('Time lag (ms)')
plt.ylabel('R2')

In [ ]:
fig, ax = plt.subplots()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
for i in range(len(pred_range_arr)):
    plt.plot(lag_axis, r2_feature_array_dynam[i,:,0].T,label=pred_range_arr[i])
plt.legend(fontsize=8)
plt.xlabel('Time lag (ms)')
plt.ylabel('R2')
plt.title('x-dir')
plt.show()

x_time_max_array = lag_axis[np.argmax(r2_feature_array_dynam[:,:,0],axis=1)]
fig, ax = plt.subplots()
plt.plot(x_time_max_array,range(len(pred_range_arr)),'k--')
plt.axvline(x=x_time_max,color='k')
plt.yticks(range(len(pred_range_arr)),['-100','0','200','400','600','800'])
plt.xlabel('Best time lag (ms)')
plt.ylabel('Time window start (ms)')

In [ ]:
fig, ax = plt.subplots()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
for i in range(len(pred_range_arr)):
    plt.plot(lag_axis, r2_feature_array_dynam[i,:,1].T,label=pred_range_arr[i])
plt.legend(fontsize=8)
plt.xlabel('Time lag (ms)')
plt.ylabel('R2')
plt.title('y-dir')

y_time_max_array = lag_axis[np.argmax(r2_feature_array_dynam[:,:,1],axis=1)]
fig, ax = plt.subplots()
plt.plot(y_time_max_array,range(len(pred_range_arr)),'k--')
plt.axvline(x=y_time_max,color='k')
plt.yticks(range(len(pred_range_arr)),['-100','0','200','400','600','800'])
plt.xlabel('Best time lag (ms)')
plt.ylabel('Time window start (ms)')

In [ ]:
n_cd_dims = 4
x_max_idx_arr = np.argmax(r2_feature_array_dynam[:,:,0],axis=1)
x_cd_dynam_weight=[]
x_fb_dynam_weight = []
for i in range(len(pred_range_arr)):
    x_cd_dynam_weight.append(coef_array_dynam[i,x_max_idx_arr[i],0,:n_cd_dims])
    x_fb_dynam_weight.append(coef_array_dynam[i,x_max_idx_arr[i],0,n_cd_dims:])
x_cd_dynam_weight = np.sum(abs(np.array((x_cd_dynam_weight))),axis=1)
x_fb_dynam_weight = np.sum(abs(np.array((x_fb_dynam_weight))),axis=1)

y_max_idx_arr = np.argmax(r2_feature_array_dynam[:,:,1],axis=1)
y_cd_dynam_weight=[]
y_fb_dynam_weight = []
for i in range(len(pred_range_arr)):
    y_cd_dynam_weight.append(coef_array_dynam[i,y_max_idx_arr[i],1,:n_cd_dims])
    y_fb_dynam_weight.append(coef_array_dynam[i,y_max_idx_arr[i],1,n_cd_dims:])
y_cd_dynam_weight = np.sum(abs(np.array((y_cd_dynam_weight))),axis=1)
y_fb_dynam_weight = np.sum(abs(np.array((y_fb_dynam_weight))),axis=1)

In [ ]:
if x_time_max == y_time_max:
    x_cd_weight = np.sum(abs(best_coef[0,:n_cd_dims]))
    x_fb_weight = np.sum(abs(best_coef[0,n_cd_dims:]))
    y_cd_weight = np.sum(abs(best_coef[1,:n_cd_dims]))
    y_fb_weight = np.sum(abs(best_coef[1,n_cd_dims:]))
else:
    x_cd_weight = np.sum(abs(x_best_coef[0,:n_cd_dims]))
    x_fb_weight = np.sum(abs(x_best_coef[0,n_cd_dims:]))
    y_cd_weight = np.sum(abs(y_best_coef[1,:n_cd_dims]))
    y_fb_weight = np.sum(abs(y_best_coef[1,n_cd_dims:]))

In [ ]:
x_axis = np.array(range(len(pred_range_arr)))
plt.bar(x_axis,x_cd_dynam_weight,width=0.2,color='green',label='CD')
plt.bar(x_axis+0.2,x_fb_dynam_weight,width=0.2,color='magenta',label='FB')
plt.axhline(x_cd_weight,color='green')
plt.axhline(x_fb_weight,color='magenta')

plt.xticks(range(len(pred_range_arr)),['-100','0','200','400','600','800'])
plt.ylabel('Sum signal dim weight in decoder')
plt.xlabel('Time window start (ms)')

In [ ]:
x_axis = np.array(range(len(pred_range_arr)))
plt.bar(x_axis+0,y_cd_dynam_weight,width=0.2,color='green',label='CD')
plt.bar(x_axis+0.2,y_fb_dynam_weight,width=0.2,color='magenta',label='FB')
plt.axhline(y_cd_weight,color='green')
plt.axhline(y_fb_weight,color='magenta')
plt.xticks(range(len(pred_range_arr)),['-100','0','200','400','600','800'])
plt.ylabel('Sum signal weight in decoder')
plt.xlabel('Time window start (ms)')

## Cross decoding

In [ ]:
monkey = "Han_20171207"
# monkey = 'Duncan_20190710'
data = np.load(monkey+'_ac150_spikes_hand_vel_act_r2s.npz')
data.files

In [ ]:
lag_axis = np.arange(-300,300,20)
print(lag_axis[np.argmax(data['x_r2_cd_only'])])
print(lag_axis[np.argmax(data['x_r2_fb_only'])])
# print(lag_axis[np.argmax(data['x_r2_cd_fb'])])
print(lag_axis[np.argmax(data['y_r2_cd_only'])])
print(lag_axis[np.argmax(data['y_r2_fb_only'])])
# print(lag_axis[np.argmax(data['y_r2_cd_fb'])])
print()
print(lag_axis[np.argmax(data['r2_cd_only'])])
print(lag_axis[np.argmax(data['r2_fb_only'])])

In [ ]:
dataset.data.keys().unique(0)

In [ ]:
x_field = 'spikes_smth_40'
data = np.load(monkey+'_X_cdfb_data_proj_out.npz')
data.files
dataset.add_continuous_data(data['CD_FB_proj'],'CD_FB_proj')
dataset.add_continuous_data(data['CD_proj'],'CD_proj')
dataset.add_continuous_data(data['FB_proj'],'FB_proj')
dataset.data.keys().unique(0)

In [ ]:

negative_lag = -180
positive_lag = 60
both_lag = 0
cond_dict = active_cond_dict
n_trials = active_n_trials

y_field ='hand_vel'
train_range = (-100,1000)
train_pos_lag_range = (train_range[0]+positive_lag, train_range[1]+positive_lag)
train_neg_lag_range = (train_range[0]+negative_lag, train_range[1]+negative_lag)
train_both_lag_range = (train_range[0]+both_lag, train_range[1]+both_lag)
n_timepoints = int((train_range[1] - train_range[0])/dataset.bin_width)

train_mask = active_mask
# _, eff_weights, eff_offset, _,_ = fit_and_predict(dataset, train_mask, 'move_onset_time',train_range, negative_lag, 'CD_proj', y_field, cond_dict = cond_dict)
# _, _, _,_,act_eff_vel_df = pred_with_new_weights(dataset, train_mask, 'move_onset_time',train_range, 0,'CD_proj',
#                                                  y_field, eff_weights, eff_offset, 'move_onset_time',train_range, train_neg_lag_range, train_mask)

# _, aff_weights, aff_offset, _,_ = fit_and_predict(dataset, train_mask, 'move_onset_time',train_range, positive_lag, 'FB_proj', y_field, cond_dict = cond_dict)
# _, _, _,_,act_aff_vel_df = pred_with_new_weights(dataset, train_mask, 'move_onset_time',train_range, 0,'FB_proj',
#                                                  y_field, aff_weights, aff_offset, 'move_onset_time',train_range, train_pos_lag_range, train_mask)

_, all_weights, all_offset, _,_ = fit_and_predict(dataset, train_mask, 'move_onset_time',train_range, both_lag, 'spikes_smth_40', y_field, cond_dict = cond_dict)
_, _, _,_,act_all_vel_df = pred_with_new_weights(dataset, train_mask, 'move_onset_time',train_range, 0,'spikes_smth_40',
                                                 y_field, all_weights, all_offset, 'move_onset_time',train_range, train_both_lag_range, train_mask)

In [ ]:
df = dataset_10ms.make_trial_data(align_field='move_onset_time', align_range=train_range, ignored_trials=~active_mask, allow_overlap=True)
latents = np.array(df.CD_FB_proj).reshape(n_trials, n_timepoints, -1)
latents.shape

In [ ]:
predictions = np.array(act_all_vel_df.pred_vel).reshape(active_n_trials, n_timepoints, -1)
predictions.shape

In [ ]:
from Neural_Decoding.decoders import DenseNNDecoder
Y = predictions
data_field = 'CD_FB_proj'
lag_axis = np.arange(-100,200,50)
regressed_r2_array = nans([len(lag_axis)])
for i in range(len(lag_axis)):
    lag = lag_axis[i]
    print(lag)
    lag_train_range = (train_range[0]+lag, train_range[1]+lag)
    df = dataset_10ms.make_trial_data(align_field='move_onset_time', align_range=lag_train_range, ignored_trials=~active_mask, allow_overlap=True)
    latents = np.array(df[data_field]).reshape(n_trials, n_timepoints, -1)
    X = X = latents 
    skf = StratifiedKFold(n_splits=5,shuffle=True,random_state = 42)   
    true_concat = nans([n_trials*n_timepoints,predictions.shape[-1]])
    pred_concat = nans([n_trials*n_timepoints,predictions.shape[-1]])
    trial_save_idx = 0
    for training_set, test_set in skf.split(range(0,n_trials),cond_dict):
        #split training and testing by trials
        X_train, X_test, y_train, y_test = process_train_test(X,Y,training_set,test_set)
        # lr = LinearRegression().fit(X_train, y_train)
        # y_test_predicted = lr.predict(X_test)
        dnn = DenseNNDecoder(units=400,dropout=0.25,num_epochs=10)
        dnn.fit(X_train, y_train)
        y_test_predicted = dnn.predict(X_test)

        n = y_test_predicted.shape[0]
        true_concat[trial_save_idx:trial_save_idx+n,:] = y_test
        pred_concat[trial_save_idx:trial_save_idx+n,:] = y_test_predicted
        trial_save_idx += n
        
    sses =get_sses_pred(true_concat,pred_concat)
    sses_mean=get_sses_mean(true_concat)
    R2 =1-np.sum(sses)/np.sum(sses_mean)  
    regressed_r2_array[i] = R2
    print(R2)

In [ ]:
plt.plot(lag_axis,regressed_r2_array)
plt.ylabel('R2')
plt.xlabel('Latent lag relative to prediction')

In [ ]:
import pandas as pd
best_lag = lag_axis[np.argmax(regressed_r2_array)]
lag_train_range = (train_range[0]+best_lag, train_range[1]+best_lag)
df = dataset_10ms.make_trial_data(align_field='move_onset_time', align_range=lag_train_range, ignored_trials=~active_mask, allow_overlap=True)
X = np.array(df[data_field])
Y = np.array(act_all_vel_df.pred_vel)
lr = LinearRegression().fit(X, Y)
pred = lr.predict(X)
act_all_vel_df = pd.concat([act_all_vel_df, pd.DataFrame(pred, columns=dataset._make_midx('regr_vel_dnn_CDFB', ['x', 'y'], 2))], axis=1)

In [ ]:
df = act_all_vel_df
# Prepare for plotting
plot_dir = [0.0, 90.0, 180.0, 270.0] # limit plot directions to reduce cluttering
colors = ['red', 'blue', 'green', 'orange']
plot_dim = 'x' # plot x velocity 

x_axis = np.arange(-100,1000,dataset.bin_width)
fig, ax = plt.subplots()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
for trial_dir, color in zip(plot_dir, colors):
    cond_ids = dataset.trial_info[dataset.trial_info.cond_dir %360 == trial_dir].trial_id
    for _, trial in df[np.isin(df.trial_id, cond_ids)].groupby('trial_id'):
        plt.plot(x_axis, trial[y_field][plot_dim], color=color, linewidth=0.5)
        # plt.plot(x_axis, trial[y_field].to_numpy()[:,0], color=color, linewidth=0.5)
plt.xlabel('Time after movement onset (ms)')
# plt.ylabel('Hand velocity (cm/s)')

# plt.xlim([-100,500])
# plt.ylim([-0.65,0.65])
plt.tight_layout()
# plt.savefig(figDir + monkey + label + 'true.pdf', dpi = 'figure')
plt.show()

fig, ax = plt.subplots()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
for trial_dir, color in zip(plot_dir, colors):
    cond_ids = dataset.trial_info[dataset.trial_info.cond_dir %360 == trial_dir].trial_id
    for _, trial in df[np.isin(df.trial_id, cond_ids)].groupby('trial_id'):
         plt.plot(x_axis, trial['pred_vel'][plot_dim], color=color, linewidth=0.5)
        # plt.plot(x_axis, trial.pred_vel.to_numpy()[:,0], color=color, linewidth=0.5)
plt.xlabel('Time after movement onset (ms)')
# plt.ylabel('Hand velocity (cm/s)')
# plt.xlim([-100,500])
# plt.ylim([-0.65,0.65])
plt.tight_layout()
# plt.savefig(figDir + monkey + label + str(0) +'_pred.pdf', dpi = 'figure')
plt.show()

fig, ax = plt.subplots()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
for trial_dir, color in zip(plot_dir, colors):
    cond_ids = dataset.trial_info[dataset.trial_info.cond_dir %360 == trial_dir].trial_id
    for _, trial in df[np.isin(df.trial_id, cond_ids)].groupby('trial_id'):
         plt.plot(x_axis, trial['regr_vel_dnn_CDFB'][plot_dim], color=color, linewidth=0.5)
        # plt.plot(x_axis, trial.pred_vel.to_numpy()[:,0], color=color, linewidth=0.5)
plt.xlabel('Time after movement onset (ms)')
# plt.ylabel('Hand velocity (cm/s)')
# plt.xlim([-100,500])
# plt.ylim([-0.65,0.65])
plt.tight_layout()
# plt.savefig(figDir + monkey + label + str(0) +'_pred.pdf', dpi = 'figure')
plt.show()

In [ ]:
y_field ='hand_vel'
lag_axis = np.arange(-300,300,20)
pred_range = (0, 120)
trial_mask = active_mask
cond_dict = active_cond_dict

# Note it differs for x- and y-dir
cd_lag = -60
fb_lag = 40
cdfb_lag = 60

_, active_cd_coef,active_cd_intercept, cd_vel_df, _ = fit_and_predict(dataset, trial_mask, 'move_onset_time',pred_range, cd_lag, 'CD_proj', y_field, cond_dict = cond_dict)
_, active_fb_coef,active_fb_intercept, fb_vel_df, _ = fit_and_predict(dataset, trial_mask, 'move_onset_time',pred_range, fb_lag, 'FB_proj', y_field, cond_dict = cond_dict)
_, active_cdfb_coef,active_cdfb_intercept, cdfb_vel_df, _ = fit_and_predict(dataset, trial_mask, 'move_onset_time',pred_range, cdfb_lag, 'CD_FB_proj', y_field, cond_dict = cond_dict)

In [ ]:
pas_cd_r2_array = nans([len(lag_axis)]); pas_cd_x_r2_array = nans([len(lag_axis)]); pas_cd_y_r2_array = nans([len(lag_axis)])
for i in range(len(lag_axis)):
    lag = lag_axis[i]
    r2, _, x_r2,y_r2,_ = pred_with_new_weights(dataset, passive_mask, 'move_onset_time',(-100,120), lag,'CD_proj',
                                                    y_field, active_cd_coef, active_cd_intercept, 'move_onset_time',pred_range, (pred_range[0]+cd_lag, pred_range[1]+cd_lag), active_mask)
    # r2, _, x_r2,y_r2,_ = pred_with_new_weights(dataset, nan_mask, 'bump_time',(0,120), lag,'CD_proj',
    #                                             y_field, active_cd_coef, active_cd_intercept, 'move_onset_time',pred_range, (pred_range[0]+cd_lag, pred_range[1]+cd_lag), active_mask)
    pas_cd_r2_array[i] = r2; pas_cd_x_r2_array[i] = x_r2; pas_cd_y_r2_array[i] = y_r2

In [ ]:
pas_fb_r2_array = nans([len(lag_axis)]); pas_fb_x_r2_array = nans([len(lag_axis)]); pas_fb_y_r2_array = nans([len(lag_axis)])
for i in range(len(lag_axis)):
    lag = lag_axis[i]
    r2, _, x_r2,y_r2,_ = pred_with_new_weights(dataset, passive_mask, 'move_onset_time',(-100,120), lag,'FB_proj',
                                                    y_field, active_fb_coef, active_fb_intercept, 'move_onset_time',pred_range, (pred_range[0]+fb_lag, pred_range[1]+fb_lag), active_mask)
    # r2, _, x_r2,y_r2,_ = pred_with_new_weights(dataset, nan_mask, 'bump_time',(0,120), lag,'FB_proj',
    #                                            y_field, active_fb_coef, active_fb_intercept, 'move_onset_time',pred_range, (pred_range[0]+fb_lag, pred_range[1]+fb_lag), active_mask)
    pas_fb_r2_array[i] = r2; pas_fb_x_r2_array[i] = x_r2; pas_fb_y_r2_array[i] = y_r2

In [ ]:
pas_cdfb_r2_array = nans([len(lag_axis)]); pas_cdfb_x_r2_array = nans([len(lag_axis)]); pas_cdfb_y_r2_array = nans([len(lag_axis)])
for i in range(len(lag_axis)):
    lag = lag_axis[i]
    r2, _, x_r2,y_r2,_ = pred_with_new_weights(dataset, passive_mask, 'move_onset_time',(-100,120), lag,'CD_FB_proj',
                                                    y_field, active_cdfb_coef, active_cdfb_intercept, 'move_onset_time',pred_range, (pred_range[0]+cdfb_lag, pred_range[1]+cdfb_lag), active_mask)
    # r2, _, x_r2,y_r2,_ = pred_with_new_weights(dataset, nan_mask, 'bump_time',(0,120), lag,'CD_FB_proj',
    #                                        y_field, active_cdfb_coef, active_cdfb_intercept, 'move_onset_time',pred_range, (pred_range[0]+cdfb_lag, pred_range[1]+cdfb_lag), active_mask)
    pas_cdfb_r2_array[i] = r2; pas_cdfb_x_r2_array[i] = x_r2; pas_cdfb_y_r2_array[i] = y_r2

In [ ]:
# np.savez(monkey+'_hand_vel_bestx_act120_pas120_r2s_cross', \
#          pas_cd_r2_array = pas_cd_r2_array, pas_cd_x_r2_array = pas_cd_x_r2_array, pas_cd_y_r2_array = pas_cd_y_r2_array, \
#          pas_fb_r2_array = pas_fb_r2_array, pas_fb_x_r2_array = pas_fb_x_r2_array, pas_fb_y_r2_array = pas_fb_y_r2_array, \
#          pas_cdfb_r2_array = pas_cdfb_r2_array, pas_cdfb_x_r2_array = pas_cdfb_x_r2_array, pas_cdfb_y_r2_array = pas_cdfb_y_r2_array)

# data = np.load(monkey+'_hand_vel_bestx_act120_pas120_r2s_cross.npz')
# data.files

np.savez(monkey+'_hand_vel_besty_act120_pas120_r2s_cross', \
         pas_cd_r2_array = pas_cd_r2_array, pas_cd_x_r2_array = pas_cd_x_r2_array, pas_cd_y_r2_array = pas_cd_y_r2_array, \
         pas_fb_r2_array = pas_fb_r2_array, pas_fb_x_r2_array = pas_fb_x_r2_array, pas_fb_y_r2_array = pas_fb_y_r2_array, \
         pas_cdfb_r2_array = pas_cdfb_r2_array, pas_cdfb_x_r2_array = pas_cdfb_x_r2_array, pas_cdfb_y_r2_array = pas_cdfb_y_r2_array)


data = np.load(monkey+'_hand_vel_besty_act120_pas120_r2s_cross.npz')
data.files

In [ ]:
lag_axis = np.arange(-300,300,20)

fig, ax = plt.subplots()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.plot(lag_axis, data['pas_cd_x_r2_array'],color = 'green',label='cd')
plt.plot(lag_axis, data['pas_fb_x_r2_array'],color = 'magenta',label='fb')
plt.plot(lag_axis, data['pas_cdfb_x_r2_array'],color = 'brown',label='cd+fb')
plt.axvline(0, color = 'k', linestyle='--')
# print(time_max)
plt.xlabel('Time lag (ms)')
plt.ylabel('R2')
plt.title(x_field)
plt.ylim([-0.7,0.7])
plt.legend()
plt.tight_layout()
# plt.savefig(figDir + monkey +'_pas_vel_x_cross.pdf', dpi = 'figure')
plt.show()

fig, ax = plt.subplots()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.plot(lag_axis, data['pas_cd_y_r2_array'],color = 'green',label='cd')
plt.plot(lag_axis, data['pas_fb_y_r2_array'],color = 'magenta',label='fb')
plt.plot(lag_axis, data['pas_cdfb_y_r2_array'],color = 'brown',label='cd+fb')
plt.axvline(0, color = 'k', linestyle='--')
# print(time_max)
plt.xlabel('Time lag (ms)')
plt.ylabel('R2')
plt.title(x_field)
plt.ylim([-0.7,0.7])
plt.legend()
plt.tight_layout()
# plt.savefig(figDir + monkey +'_pas_vel_y_cross.pdf', dpi = 'figure')
plt.show()

In [ ]:
# Cross-decoding

dataset = dataset_10ms
# all_mask = dataset.trial_info.split != 'none'


negative_lag = -100
positive_lag = 60
both_lag = 0

y_field ='hand_vel'
train_range = (-100,1000)
train_pos_lag_range = (train_range[0]+positive_lag, train_range[1]+positive_lag)
train_neg_lag_range = (train_range[0]+negative_lag, train_range[1]+negative_lag)
train_mask = active_mask
# train_mask = passive_mask
_, aff_weights, aff_offset, act_aff_vel_df,_ = fit_and_predict(dataset, train_mask, 'move_onset_time', train_range, positive_lag, 'ac150_FB_proj_spikes_smth_150_oneside', y_field)
_, eff_weights, eff_offset, act_eff_vel_df,_ = fit_and_predict(dataset, train_mask, 'move_onset_time', train_range, negative_lag,'ac150_CD_proj_spikes_smth_150_oneside',y_field)
_, both_weights, both_offset, act_both_vel_df,_ = fit_and_predict(dataset, train_mask, 'move_onset_time', train_range, both_lag,'ac150_CD_FB_proj_spikes_smth_150_oneside',y_field)

#pred active
pred_range = (-100,1000)
active_x_axis = np.arange(pred_range[0], pred_range[1], dataset.bin_width)
_, _, _,_,act_aff_vel_df = pred_with_new_weights(dataset, active_mask, 'move_onset_time',pred_range, positive_lag,'ac150_FB_proj_spikes_smth_150_oneside',
                                                 y_field, aff_weights, aff_offset, 'move_onset_time',train_range, train_pos_lag_range, train_mask)
_, _, _,_,act_eff_vel_df = pred_with_new_weights(dataset, active_mask, 'move_onset_time',pred_range, negative_lag,'ac150_CD_proj_spikes_smth_150_oneside',
                                                 y_field, eff_weights, eff_offset, 'move_onset_time',train_range, train_neg_lag_range, train_mask)
_, _, _,_,act_both_vel_df = pred_with_new_weights(dataset, active_mask, 'move_onset_time',pred_range, both_lag,'ac150_CD_FB_proj_spikes_smth_150_oneside',
                                                 y_field, both_weights, both_offset, 'move_onset_time',train_range, train_neg_lag_range, train_mask)
#pred passive
pred_range = (-100, 500)
passive_x_axis = np.arange(pred_range[0], pred_range[1], dataset.bin_width)
_, _, _,_,pas_aff_vel_df = pred_with_new_weights(dataset, passive_mask, 'move_onset_time',pred_range, positive_lag,'ac150_FB_proj_spikes_smth_150_oneside',
                                                 y_field, aff_weights, aff_offset, 'move_onset_time',train_range, train_pos_lag_range, train_mask)
_, _, _,_,pas_eff_vel_df = pred_with_new_weights(dataset, passive_mask, 'move_onset_time',pred_range, negative_lag,'ac150_CD_proj_spikes_smth_150_oneside',
                                                 y_field, eff_weights, eff_offset, 'move_onset_time',train_range, train_neg_lag_range, train_mask)
_, _, _,_,pas_both_vel_df = pred_with_new_weights(dataset, passive_mask, 'move_onset_time',pred_range, both_lag,'ac150_CD_FB_proj_spikes_smth_150_oneside',
                                                 y_field, both_weights, both_offset, 'move_onset_time',train_range, train_neg_lag_range, train_mask)

# #pred nan
# pred_range = (-100, 1000)
# passive_x_axis = np.arange(pred_range[0], pred_range[1], dataset.bin_width)
# _, _, _,_,pas_aff_vel_df = pred_with_new_weights(dataset, nan_mask, 'move_onset_time',pred_range, positive_lag,'FB_proj',
#                                                  y_field, aff_weights, aff_offset, 'move_onset_time',train_range, train_pos_lag_range, train_mask)
# _, _, _,_,pas_eff_vel_df = pred_with_new_weights(dataset, nan_mask, 'move_onset_time',pred_range, negative_lag,'CD_proj',
#                                                  y_field, eff_weights, eff_offset, 'move_onset_time',train_range, train_neg_lag_range, train_mask)
# _, _, _,_,pas_both_vel_df = pred_with_new_weights(dataset, nan_mask, 'move_onset_time',pred_range, both_lag,'CD_FB_proj',
#                                                  y_field, both_weights, both_offset, 'move_onset_time',train_range, train_neg_lag_range, train_mask)

plot_dir = [0.0, 180.0] # limit plot directions to reduce cluttering
colors = ['gray','gray']
plot_dim = 'x'
fig, axs = plt.subplots(6, 4, sharex=False, sharey=True, figsize=(18, 18))
# plt.ylim(-50,50)
i = 0
alpha = 0.5
for trial_dir, color in zip(plot_dir, colors):
    cond_ids = dataset.trial_info[dataset.trial_info.cond_dir%360 == trial_dir].trial_id
    for _, trial in act_eff_vel_df[np.isin(act_eff_vel_df.trial_id, cond_ids)].groupby('trial_id'):
        axs[0][i].plot(active_x_axis, trial[y_field][plot_dim], color=color, alpha = alpha, linewidth=0.5)
        axs[0][i].plot(active_x_axis, trial.pred_vel[plot_dim], color='green', alpha = alpha, linewidth=0.5)
        axs[0][i].axvline(x=300, ls='--')
    for _, trial in act_aff_vel_df[np.isin(act_aff_vel_df.trial_id, cond_ids)].groupby('trial_id'):
        axs[1][i].plot(active_x_axis, trial.pred_vel[plot_dim], color='magenta', alpha = alpha, linewidth=0.5)
        axs[1][i].plot(active_x_axis, trial[y_field][plot_dim], color=color, alpha = alpha, linewidth=0.5)
        axs[1][i].axvline(x=300, ls='--')
    for _, trial in act_both_vel_df[np.isin(act_both_vel_df.trial_id, cond_ids)].groupby('trial_id'):
        axs[2][i].plot(active_x_axis, trial[y_field][plot_dim], color=color, alpha = alpha, linewidth=0.5)
        axs[2][i].plot(active_x_axis, trial.pred_vel[plot_dim], color='brown', alpha = alpha, linewidth=0.5)    
        axs[2][i].spines[['right', 'top']].set_visible(False) 
        axs[2][i].axvline(x=300, ls='--')
    # cond_ids = dataset.trial_info[dataset.trial_info.bump_dir%360 == trial_dir].trial_id
    for _, trial in pas_eff_vel_df[np.isin(pas_eff_vel_df.trial_id, cond_ids)].groupby('trial_id'):
        axs[3][i].plot(passive_x_axis, trial.pred_vel[plot_dim], color='green', alpha = alpha, linewidth=0.5)
        axs[3][i].plot(passive_x_axis, trial[y_field][plot_dim], color=color, alpha = alpha, linewidth=0.5)
        axs[3][i].axvline(x=120, ls='--')
    for _, trial in pas_aff_vel_df[np.isin(pas_aff_vel_df.trial_id, cond_ids)].groupby('trial_id'):
        axs[4][i].plot(passive_x_axis, trial.pred_vel[plot_dim], color='magenta', alpha = alpha, linewidth=0.5)
        axs[4][i].plot(passive_x_axis, trial[y_field][plot_dim], color=color, alpha = alpha, linewidth=0.5)
        axs[4][i].axvline(x=120, ls='--')
    for _, trial in pas_both_vel_df[np.isin(pas_both_vel_df.trial_id, cond_ids)].groupby('trial_id'):
        axs[5][i].plot(passive_x_axis, trial.pred_vel[plot_dim], color='brown', alpha = alpha, linewidth=0.5)
        axs[5][i].plot(passive_x_axis, trial[y_field][plot_dim], color=color, alpha = alpha, linewidth=0.5)
        axs[5][i].spines[['right', 'top']].set_visible(False)
        axs[5][i].axvline(x=120, ls='--')
    i+=2


negative_lag = -100
positive_lag = 120
both_lag = 0

y_field ='hand_vel'
train_range = (-100,1000)
train_pos_lag_range = (train_range[0]+positive_lag, train_range[1]+positive_lag)
train_neg_lag_range = (train_range[0]+negative_lag, train_range[1]+negative_lag)
train_mask = active_mask
# train_mask = passive_mask

_, aff_weights, aff_offset, act_aff_vel_df,_ = fit_and_predict(dataset, train_mask, 'move_onset_time', train_range, positive_lag, 'ac150_FB_proj_spikes_smth_150_oneside', y_field)
_, eff_weights, eff_offset, act_eff_vel_df,_ = fit_and_predict(dataset, train_mask, 'move_onset_time', train_range, negative_lag,'ac150_CD_proj_spikes_smth_150_oneside',y_field)
_, both_weights, both_offset, act_both_vel_df,_ = fit_and_predict(dataset, train_mask, 'move_onset_time', train_range, both_lag,'ac150_CD_FB_proj_spikes_smth_150_oneside',y_field)

#pred active
pred_range = (-100, 1000)
active_x_axis = np.arange(pred_range[0], pred_range[1], dataset.bin_width)
_, _, _,_,act_aff_vel_df = pred_with_new_weights(dataset, active_mask, 'move_onset_time',pred_range, positive_lag,'ac150_FB_proj_spikes_smth_150_oneside',
                                                 y_field, aff_weights, aff_offset, 'move_onset_time',train_range, train_pos_lag_range, train_mask)
_, _, _,_,act_eff_vel_df = pred_with_new_weights(dataset, active_mask, 'move_onset_time',pred_range, negative_lag,'ac150_CD_proj_spikes_smth_150_oneside',
                                                 y_field, eff_weights, eff_offset, 'move_onset_time',train_range, train_neg_lag_range, train_mask)
_, _, _,_,act_both_vel_df = pred_with_new_weights(dataset, active_mask, 'move_onset_time',pred_range, both_lag,'ac150_CD_FB_proj_spikes_smth_150_oneside',
                                                 y_field, both_weights, both_offset, 'move_onset_time',train_range, train_neg_lag_range, train_mask)
#pred passive
pred_range = (-100, 500)
passive_x_axis = np.arange(pred_range[0], pred_range[1], dataset.bin_width)
_, _, _,_,pas_aff_vel_df = pred_with_new_weights(dataset, passive_mask, 'move_onset_time',pred_range, positive_lag,'ac150_FB_proj_spikes_smth_150_oneside',
                                                 y_field, aff_weights, aff_offset, 'move_onset_time',train_range, train_pos_lag_range, train_mask)
_, _, _,_,pas_eff_vel_df = pred_with_new_weights(dataset, passive_mask, 'move_onset_time',pred_range, negative_lag,'ac150_CD_proj_spikes_smth_150_oneside',
                                                 y_field, eff_weights, eff_offset, 'move_onset_time',train_range, train_neg_lag_range, train_mask)
_, _, _,_,pas_both_vel_df = pred_with_new_weights(dataset, passive_mask, 'move_onset_time',pred_range, both_lag,'ac150_CD_FB_proj_spikes_smth_150_oneside',
                                                 y_field, both_weights, both_offset, 'move_onset_time',train_range, train_neg_lag_range, train_mask)

#pred nan
# pred_range = (-100, 1000)
# passive_x_axis = np.arange(pred_range[0], pred_range[1], dataset.bin_width)
# _, _, _,_,pas_aff_vel_df = pred_with_new_weights(dataset, nan_mask, 'move_onset_time',pred_range, positive_lag,'FB_proj',
#                                                  y_field, aff_weights, aff_offset, 'move_onset_time',train_range, train_pos_lag_range, train_mask)
# _, _, _,_,pas_eff_vel_df = pred_with_new_weights(dataset, nan_mask, 'move_onset_time',pred_range, negative_lag,'CD_proj',
#                                                  y_field, eff_weights, eff_offset, 'move_onset_time',train_range, train_neg_lag_range, train_mask)
# _, _, _,_,pas_both_vel_df = pred_with_new_weights(dataset, nan_mask, 'move_onset_time',pred_range,both_lag,'CD_FB_proj',
#                                                  y_field, both_weights, both_offset, 'move_onset_time',train_range, train_neg_lag_range, train_mask)


plot_dir = [90.0, 270.0] # limit plot directions to reduce cluttering
colors = ['gray', 'gray']
plot_dim = 'y'
i = 1
alpha = 0.5
for trial_dir, color in zip(plot_dir, colors):
    cond_ids = dataset.trial_info[dataset.trial_info.cond_dir%360 == trial_dir].trial_id
    for _, trial in act_eff_vel_df[np.isin(act_eff_vel_df.trial_id, cond_ids)].groupby('trial_id'):
        axs[0][i].plot(active_x_axis, trial[y_field][plot_dim], color=color, alpha = alpha, linewidth=0.5)
        axs[0][i].plot(active_x_axis, trial.pred_vel[plot_dim], color='green', alpha = alpha, linewidth=0.5)
        axs[0][i].axvline(x=300, ls='--')
    for _, trial in act_aff_vel_df[np.isin(act_aff_vel_df.trial_id, cond_ids)].groupby('trial_id'):
        axs[1][i].plot(active_x_axis, trial.pred_vel[plot_dim], color='magenta', alpha = alpha, linewidth=0.5)
        axs[1][i].plot(active_x_axis, trial[y_field][plot_dim], color=color, alpha = alpha, linewidth=0.5)
        axs[1][i].axvline(x=300, ls='--')
    for _, trial in act_both_vel_df[np.isin(act_both_vel_df.trial_id, cond_ids)].groupby('trial_id'):
        axs[2][i].plot(active_x_axis, trial[y_field][plot_dim], color=color, alpha = alpha, linewidth=0.5)
        axs[2][i].plot(active_x_axis, trial.pred_vel[plot_dim], color='brown', alpha = alpha, linewidth=0.5)    
        axs[2][i].spines[['right', 'top']].set_visible(False) 
        axs[2][i].axvline(x=300, ls='--')
    # cond_ids = dataset.trial_info[dataset.trial_info.bump_dir%360 == trial_dir].trial_id
    for _, trial in pas_eff_vel_df[np.isin(pas_eff_vel_df.trial_id, cond_ids)].groupby('trial_id'):
        axs[3][i].plot(passive_x_axis, trial.pred_vel[plot_dim], color='green', alpha = alpha, linewidth=0.5)
        axs[3][i].plot(passive_x_axis, trial[y_field][plot_dim], color=color, alpha = alpha, linewidth=0.5)
        axs[3][i].axvline(x=120, ls='--')
    for _, trial in pas_aff_vel_df[np.isin(pas_aff_vel_df.trial_id, cond_ids)].groupby('trial_id'):
        axs[4][i].plot(passive_x_axis, trial.pred_vel[plot_dim], color='magenta', alpha = alpha, linewidth=0.5)
        axs[4][i].plot(passive_x_axis, trial[y_field][plot_dim], color=color, alpha = alpha, linewidth=0.5)
        axs[4][i].axvline(x=120, ls='--')
    for _, trial in pas_both_vel_df[np.isin(pas_both_vel_df.trial_id, cond_ids)].groupby('trial_id'):
        axs[5][i].plot(passive_x_axis, trial.pred_vel[plot_dim], color='brown', alpha = alpha, linewidth=0.5)
        axs[5][i].plot(passive_x_axis, trial[y_field][plot_dim], color=color, alpha = alpha, linewidth=0.5)
        axs[5][i].spines[['right', 'top']].set_visible(False)
        axs[5][i].axvline(x=120, ls='--')
    i+=2

fig.supxlabel('Time after movement onset (ms)')
# axs[0][0].set_ylabel('Prediction',fontsize=14)
# axs[1][0].set_ylabel('Prediction',fontsize=14)
# axs[0][0].set_ylabel('Hand acceleration \n (cm/s^2)',fontsize=14)
# axs[1][0].set_ylabel('Hand acceleration \n (cm/s^2)',fontsize=14)
axs[0][0].set_ylabel('Hand velocity \n (cm/s)',fontsize=14)
axs[1][0].set_ylabel('Hand velocity \n (cm/s)',fontsize=14)


axs[0][0].set_title('0 deg')
axs[0][1].set_title('90 deg')
axs[0][2].set_title('180 deg')
axs[0][3].set_title('270 deg')


# legend_elements = [Patch(facecolor='magenta', label='Afferent prediction'),
#                     Patch(facecolor='k', label='Efferent prediction')]
# legend_elements = [Patch(facecolor='magenta', label='Afferent prediction')]
# plt.legend(handles=legend_elements)
plt.tight_layout()
# figDir = '/Users/sherryan/area2_population_analysis/'
# plt.savefig(figDir + monkey + '_cross_vel_both_early_aligned.pdf', dpi = 'figure')
plt.show()

## Neuron weights

### PC proj_back

In [ ]:
data = np.load(monkey+'_CDFB_weights_'+'PCA_40'+'.npz')
X = data['CD_axes']
eff_weights = pca.inverse_transform(X)
print(eff_weights.shape)
X = data['FB_axes']
aff_weights = pca.inverse_transform(X)
print(aff_weights.shape)
np.savez(monkey+'_CDFB_weights_pc_proj_back', CD_axes = eff_weights, FB_axes = aff_weights) 

In [ ]:
aff_weights_mean = np.mean(abs(aff_weights),axis=0)
print(aff_weights_mean.shape)
plt.hist(aff_weights_mean)
plt.show()
eff_weights_mean = np.mean(abs(eff_weights),axis=0)
print(eff_weights_mean.shape)
plt.hist(eff_weights_mean)

In [ ]:
#Neuron weights plot

def adjacent_values(vals, q1, q3):
    upper_adjacent_value = q3 + (q3 - q1) * 1.5
    upper_adjacent_value = np.clip(upper_adjacent_value, q3, vals[-1])
    lower_adjacent_value = q1 - (q3 - q1) * 1.5
    lower_adjacent_value = np.clip(lower_adjacent_value, vals[0], q1)
    return lower_adjacent_value, upper_adjacent_value
    
plt.hist(aff_weights_mean)
plt.show()
plt.hist(eff_weights_mean)
plt.show()

Ki_x = []
for i in range(n_neurons):
    Wa = aff_weights_mean[i]
    We = eff_weights_mean[i]
    if abs(Wa) > 0.02 or abs(We) > 0.02:
        Ki_x.append((abs(Wa) - abs(We)) / (abs(Wa)+abs(We)))
    else:
        Ki_x.append(np.nan)
print(len(Ki_x))

Ki_x = np.array(Ki_x)
Ki_x_plot = Ki_x[~np.isnan(Ki_x)]
print(len(Ki_x_plot))

fig, ax = plt.subplots(figsize=(5, 4))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
# fig.suptitle('Relative contribution to acc decoder between Afference and Efference')
import seaborn as sns
parts = ax.violinplot(Ki_x_plot,showmeans = False, showextrema=False)
for pc in parts['bodies']:
    pc.set_facecolor('grey')
    pc.set_edgecolor('black')
    pc.set_alpha(.5)


quartile1, medians, quartile3 = np.percentile(Ki_x_plot, [25, 50, 75])
whiskers = adjacent_values(sorted(Ki_x_plot), quartile1, quartile3)
whiskers_min, whiskers_max = whiskers[0], whiskers[1]
ax.scatter(1, medians, marker='o', color='white', s=20, zorder=3)
ax.vlines(1, quartile1, quartile3, color='k', linestyle='-', lw=5)
ax.vlines(1, whiskers_min, whiskers_max, color='k', linestyle='-', lw=1)

ax.set_ylabel('Relative contribution')
ax.set_xticks([])
# ax.set_xlabel('All neurons')
ax.set_ylim([-1,1])
plt.tight_layout()
# plt.savefig(figDir + monkey + '_contrib_violin_vel.pdf', dpi = 'figure')
plt.show()

fig, ax = plt.subplots(figsize=(5, 4))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.hist(sorted(Ki_x_plot),10,rwidth=0.8,color = 'grey')
plt.xlabel('Relative contribution')
plt.ylabel('Neuron count')
plt.xlim([-1.01,1.01])
plt.tight_layout()
# plt.savefig(figDir + monkey + '_contrib_bar_vel.pdf', dpi = 'figure')
plt.show()

In [ ]:
# np.argwhere(np.isnan(np.array(Ki_x))).squeeze()
# np.argsort(np.array(Ki_x))
valid_sort = np.array([x for x in np.argsort(np.array(Ki_x)) if x not in np.argwhere(np.isnan(np.array(Ki_x))).squeeze()])
valid_sort

#To pick exammple single neuron 
# np.array(Ki_x)[47]

np.savez(monkey+'_cdfb_contrib'+'_pc_proj', relative_contrib = np.array(Ki_x), valid_sort=valid_sort)
data = np.load(monkey+'_cdfb_contrib_pc_proj.npz')
data.files

### Neurons

In [ ]:
monkey = 'Duncan_20190710'
# monkey = 'Han_20171207'
data = np.load(monkey+'_8020_cdfb_weights_'+'spikes_smth_40'+'.npz')
eff_weights = data['CD_axes']
print(eff_weights.shape)
aff_weights = data['FB_axes']
print(aff_weights.shape)

n_neurons = dataset_10ms.data.spikes.shape[1]
print(n_neurons,'neurons')

In [ ]:
aff_weights_mean = np.mean(abs(aff_weights),axis=0)
print(aff_weights_mean.shape)
plt.hist(aff_weights_mean)
plt.show()
eff_weights_mean = np.mean(abs(eff_weights),axis=0)
print(eff_weights_mean.shape)
plt.hist(eff_weights_mean)

In [ ]:
#Neuron weights plot

def adjacent_values(vals, q1, q3):
    upper_adjacent_value = q3 + (q3 - q1) * 1.5
    upper_adjacent_value = np.clip(upper_adjacent_value, q3, vals[-1])
    lower_adjacent_value = q1 - (q3 - q1) * 1.5
    lower_adjacent_value = np.clip(lower_adjacent_value, vals[0], q1)
    return lower_adjacent_value, upper_adjacent_value
    
plt.hist(aff_weights_mean)
plt.show()
plt.hist(eff_weights_mean)
plt.show()

Ki_x = []
for i in range(n_neurons):
    Wa = aff_weights_mean[i]
    We = eff_weights_mean[i]
    if abs(Wa) > 0.1 or abs(We) > 0.1:
        Ki_x.append((abs(Wa) - abs(We)) / (abs(Wa)+abs(We)))
    else:
        Ki_x.append(np.nan)
print(len(Ki_x))

Ki_x = np.array(Ki_x)
Ki_x_plot = Ki_x[~np.isnan(Ki_x)]
print(len(Ki_x_plot))

fig, ax = plt.subplots(figsize=(5, 4))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
# fig.suptitle('Relative contribution to acc decoder between Afference and Efference')
import seaborn as sns
parts = ax.violinplot(Ki_x_plot,showmeans = False, showextrema=False)
for pc in parts['bodies']:
    pc.set_facecolor('grey')
    pc.set_edgecolor('black')
    pc.set_alpha(.5)


quartile1, medians, quartile3 = np.percentile(Ki_x_plot, [25, 50, 75])
whiskers = adjacent_values(sorted(Ki_x_plot), quartile1, quartile3)
whiskers_min, whiskers_max = whiskers[0], whiskers[1]
ax.scatter(1, medians, marker='o', color='white', s=20, zorder=3)
ax.vlines(1, quartile1, quartile3, color='k', linestyle='-', lw=5)
ax.vlines(1, whiskers_min, whiskers_max, color='k', linestyle='-', lw=1)

ax.set_ylabel('Relative contribution')
ax.set_xticks([])
# ax.set_xlabel('All neurons')
ax.set_ylim([-1,1])
plt.tight_layout()
# plt.savefig(figDir + monkey + '_contrib_violin_vel.pdf', dpi = 'figure')
plt.show()

fig, ax = plt.subplots(figsize=(5, 4))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.hist(sorted(Ki_x_plot),10,rwidth=0.8,color = 'grey')
plt.xlabel('Relative contribution')
plt.ylabel('Neuron count')
plt.xlim([-1.01,1.01])
plt.tight_layout()
# plt.savefig(figDir + monkey + '_contrib_bar_vel.pdf', dpi = 'figure')
plt.show()

In [ ]:
# np.argwhere(np.isnan(np.array(Ki_x))).squeeze()
# np.argsort(np.array(Ki_x))
valid_sort = np.array([x for x in np.argsort(np.array(Ki_x)) if x not in np.argwhere(np.isnan(np.array(Ki_x))).squeeze()])
print(valid_sort)
print(len(valid_sort),'neurons')

#To pick exammple single neuron 
print(np.array(Ki_x)[47])
print(np.array(Ki_x)[69])
print(np.array(Ki_x)[89])
print(np.array(Ki_x)[0])

# np.savez(monkey+'_cdfb_contrib'+'_neuron', relative_contrib = np.array(Ki_x), valid_sort=valid_sort)
# data = np.load(monkey+'_cdfb_contrib_neuron.npz')
# data.files